In [1]:
!pip install rouge-score
!pip install bert-score
!pip install jiwer

  Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
  Using cached transformers-4.33.2-py3-none-any.whl (7.6 MB)
  Using cached huggingface_hub-0.17.2-py3-none-any.whl (294 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached safetensors-0.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached jiwer-3.0.3-py3-none-any.whl (21 kB)
  Using cached rapidfuzz-3.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)


In [8]:
!pip install nltk

In [9]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
from rouge_score import rouge_scorer
# from baseline_lpp.baseline import run_baseline
# from utils.paths import *
from jiwer import wer
from bert_score import BERTScorer
import socket
from urllib3.connection import HTTPConnection
import nltk

In [10]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:


"""
BERTScore (https://arxiv.org/abs/1904.09675)
"""
class BERTSCORE(object):
    """
    copied from https://github.com/HuthLab/semantic-decoding/blob/main/decoding/utils_eval.py
    """
    def __init__(self, idf_sents=None, rescale = True, score = "f"):
        self.metric = BERTScorer(lang = "en", rescale_with_baseline = rescale, idf = (idf_sents is not None), idf_sents = idf_sents)
        if score == "precision": self.score_id = 0
        elif score == "recall": self.score_id = 1
        else: self.score_id = 2

    def score(self, ref, pred):
        ref_strings = [" ".join(x) for x in ref]
        pred_strings = [" ".join(x) for x in pred]
        return self.metric.score(cands = pred_strings, refs = ref_strings,verbose=True)[self.score_id].numpy()


def create_metrics_df(ground_truth: list[str], predicted: list[str], include_bert_score = False) -> pd.DataFrame:
    """
    calculates metrics
    :param ground_truth:
    :param predicted:
    :return:
    """
    # set higher timeout mb to prevent connection timeout when downloading pytorch_model.bin
    HTTPConnection.default_socket_options = (
            HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_SNDBUF, 2000000),
        (socket.SOL_SOCKET, socket.SO_RCVBUF, 2000000)
    ])

    # Initialize ROUGE scorer
    ROUGE_SCORER = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    if include_bert_score:
        BERT_SCORER = BERTSCORE()
    # Initialize variables to accumulate scores
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []
    meteor_scores = []
    wer_scores = []
    # bert_scores = []

    # Loop through each pair of ground truth and predicted sentences
    for gt, pred in zip(ground_truth, predicted):
        # Tokenize the sentences
        gt_tokens = gt.split()
        pred_tokens = pred.split()

        # Calculate ROUGE scores
        rouge_scores = ROUGE_SCORER.score(' '.join(gt_tokens), ' '.join(pred_tokens))
        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

        # Calculate BLEU score
        bleu_score = sentence_bleu(gt_tokens, pred_tokens)
        bleu_scores.append(bleu_score)

        # Calculate METEOR score
        meteor_score_value = single_meteor_score(gt_tokens, pred_tokens)
        meteor_scores.append(meteor_score_value)

        # Calculate word error rate (WER)
        wer_score = wer(reference=gt,hypothesis=pred)
        wer_scores.append(wer_score)

    # TODO: add BERTScore
    # add bert score if specified
    if include_bert_score:
        bert_scores = BERT_SCORER.score(ref=ground_truth, pred=predicted)
        # bert_scores.append(bs)

    df_scores = pd.DataFrame(
        {
            'gt': ground_truth,
            'pred': predicted,
            'rouge1': rouge1_scores,
            'rouge2': rouge2_scores,
            'rougeL': rougeL_scores,
            'bleu': bleu_scores,
            'meteor': meteor_scores,
            'wer':wer_scores,
            'bert_score': bert_scores

        }
    )
    return df_scores


# def save_baseline_metrics(include_bert_score:bool=False):
#     preproc_sentences_base, gen_sentences_base = run_baseline()
#     df_metrics_baseline = create_metrics_df(ground_truth=preproc_sentences_base, predicted=gen_sentences_base, include_bert_score=include_bert_score)
#     filename = 'baseline_metrics.csv'
#     save_path = os.path.join(eval_path, 'metrics')
#     file_path = os.path.join(save_path, filename)
#     if not os.path.exists(save_path):
#         os.makedirs(save_path)
#     df_metrics_baseline.to_csv(file_path, index=False)
#     print(f'saved {file_path}')

# if __name__ == "__main__":
#     save_baseline_metrics(include_bert_score =True)
#     print('done')


# load baseline pred


In [4]:
def load_baseline_pred():
  preproc_sentences = ['once when i was six years old i saw a magnificent picture in a book called true stories from nature about the primeval forest', 'it was a picture of a boa constrictor in the act of swallowing an animal', 'here is a copy of the drawing', 'in the book it said boa constrictors swallow their prey whole without chewing it', 'after that they are not able to move and they sleep through the six months that they need for digestion i pondered deeply then over the adventures of the jungle', 'and after some work with a coloured pencil i succeeded in making my first drawing', 'my drawing number one', 'it looked like this i showed my masterpiece to the grownups and asked them whether the drawing frightened them', 'but they answered frighten', 'why should any one be frightened by a hat my drawing was not a picture of a hat', 'it was a picture of a boa constrictor digesting an elephant', 'but since the grown ups were not able to understand it i made another drawing i drew the inside of the boa constrictor so that the grownups could see it clearly', 'they always need to have things explained', 'my drawing number two looked like this the grownups response this time was to advise me to lay aside my drawings of boa constrictors whether from the inside or the outside and devote myself instead to geography history arithmetic and grammar', 'that is why at the age of six i gave up what might have been a magnificent career as a painter', 'i had been disheartened by the failure of my drawing number one and my drawing number two', 'grownups never understand anything by themselves and it is tiresome for children to be always and forever explaining things to them', 'so then i chose another profession and learned to pilot airplanes', 'i have flown a little over all parts of the world and it is true that geography has been very useful to me', 'at a glance i can distinguish china from arizona', 'if one gets lost in the night such knowledge is valuable', 'in the course of this life i have had a great many encounters with a great many people who have been concerned with matters of consequence', 'i have lived a great deal among grownups', 'i have seen them intimately close at hand', 'and that hasnt much improved my opinion of them', 'whenever i met one of them who seemed to me at all clearsighted i tried the experiment of showing him my drawing number one which i have always kept', 'i would try to find out so if this was a person of true understanding', 'but whoever it was he or she would always say that is a hat then i would never talk to that person about boa constrictors or primeval forests or stars', 'i would bring myself down to his level', 'i would talk to him about bridge and golf and politics and neckties', 'and the grownup would be greatly pleased to have met such a sensible man', 'so i lived my life alone without anyone that i could really talk to until i had an accident with my plane in the desert of sahara six years ago', 'something was broken in my engine', 'and as i had with me neither a mechanic nor any passengers i set myself to attempt the difficult repairs all alone', 'it was a question of life or death for me i had scarcely enough drinking water to last a week', 'the first night then i went to sleep on the sand a thousand miles from any human habitation', 'i was more isolated than a shipwrecked sailor on a raft in the middle of the ocean', 'thus you can imagine my amazement at sunrise when i was awakened by an odd little voice', 'it said if you please draw me a sheep what draw me a sheep i jumped to my feet completely thunderstruck', 'i blinked my eyes hard', 'i looked carefully all around me', 'and i saw a most extraordinary small person who stood there examining me with great seriousness', 'here you may see the best portrait that later i was able to make of him', 'but my drawing is certainly very much less charming than its model', 'that however is not my fault', 'the grownups discouraged me in my painters career when i was six years old and i never learned to draw anything except boas from the outside and boas from the inside', 'now i stared at this sudden apparition with my eyes fairly starting out of my head in astonishment', 'remember i had crashed in the desert a thousand miles from any inhabited region', 'and yet my little man seemed neither to be straying uncertainly among the sands nor to be fainting from fatigue or hunger or thirst or fear', 'nothing about him gave any suggestion of a child lost in the middle of the desert a thousand miles from any human habitation', 'when at last i was able to speak i said to him but what are you doing here and in answer he repeated very slowly as if he were speaking of a matter of great consequence if you please draw me a sheep when a mystery is too overpowering one dare not disobey', 'absurd as it might seem to me a thousand miles from any human habitation and in danger of death i took out of my pocket a sheet of paper and my fountain pen', 'but then i remembered how my studies had been concentrated on geography history arithmetic and grammar and i told the little chap a little crossly too that i did not know how to draw', 'he answered me that doesnt matter', 'draw me a sheep but i had never drawn a sheep', 'so i drew for him one of the two pictures i had drawn so often', 'it was that of the boa constrictor from the outside', 'and i was astounded to hear the little fellow greet it with no no no', 'i do not want an elephant inside a boa constrictor', 'a boa constrictor is a very dangerous creature and an elephant is very cumbersome', 'where i live everything is very small', 'what i need is a sheep', 'draw me a sheep', 'so then i made a drawing', 'he looked at it carefully then he said no', 'this sheep is already very sickly', 'make me another so i made another drawing', 'my friend smiled gently and indulgently', 'you see yourself he said that this is not a sheep', 'this is a ram', 'it has horns', 'so then i did my drawing over once more', 'but it was rejected too just like the others', 'this one is too old', 'i want a sheep that will live a long time', 'by this time my patience was exhausted because i was in a hurry to start taking my engine apart', 'so i tossed off this drawing', 'and i threw out an explanation with it', 'this is only his box', 'the sheep you asked for is inside i was very surprised to see a light break over the face of my young judge that is exactly the way i wanted it', 'do you think that this sheep will have to have a great deal of grass why because where i live everything is very small there will surely be enough grass for him i said', 'it is a very small sheep that i have given you he bent his head over the drawing not so small that look', 'he has gone to sleep and that is how i made the acquaintance of the little prince', 'it took me a long time to learn where he came from', 'the little prince who asked me so many questions never seemed to hear the ones i asked him', 'it was from words dropped by chance that little by little everything was revealed to me', 'the first time he saw my airplane for instance i shall not draw my airplane that would be much too complicated for me he asked me what is that object that is not an object', 'it flies', 'it is an airplane', 'it is my airplane and i was proud to have him learn that i could fly', 'he cried out then what', 'you dropped down from the sky yes i answered modestly', 'oh', 'that is funny and the little prince broke into a lovely peal of laughter which irritated me very much', 'i like my misfortunes to be taken seriously', 'then he added so you too come from the sky', 'which is your planet at that moment i caught a gleam of light in the impenetrable mystery of his presence and i demanded abruptly do you come from another planet but he did not reply', 'he tossed his head gently without taking his eyes from my plane it is true that on that you cant have come from very far away and he sank into a reverie which lasted a long time', 'then taking my sheep out of his pocket he buried himself in the contemplation of his treasure', 'you can imagine how my curiosity was aroused by this halfconfidence about the other planets i made a great effort therefore to find out more on this subject', 'my little man where do you come from', 'what is this where i live of which you speak', 'where do you want to take your sheep after a reflective silence he answered the thing that is so good about the box you have given me is that at night he can use it as his house that is so', 'and if you are good i will give you a string too so that you can tie him during the day and a post to tie him to but the little prince seemed shocked by this offer tie him', 'what a queer idea but if you dont tie him i said he will wander off somewhere and get lost my friend broke into another peal of laughter but where do you think he would go anywhere', 'straight ahead of him then the little prince said earnestly that doesnt matter', 'where i live everything is so small and with perhaps a hint of sadness he added straight ahead of him nobody can go very far i had thus learned a second fact of great importance this was that the planet the little prince came from was scarcely any larger than a house', 'but that did not really surprise me much', 'i knew very well that in addition to the great planets such as the earth jupiter mars venus to which we have given names there are also hundreds of others some of which are so small that one has a hard time seeing them through the telescope', 'when an astronomer discovers one of these he does not give it a name but only a number', 'he might call it for example asteroid  i have serious reason to believe that the planet from which the little prince came is the asteroid known as b', 'this asteroid has only once been seen through the telescope', 'that was by a turkish astronomer in ', 'on making his discovery the astronomer had presented it to the international astronomical congress in a great demonstration', 'but he was in turkish costume and so nobody would believe what he said', 'grownups are like that fortunately however for the reputation of asteroid b a turkish dictator made a law that his subjects under pain of death should change to european costume', 'so in  the astronomer gave his demonstration all over again dressed with impressive style and elegance', 'and this time everybody accepted his report', 'if i have told you these details about the asteroid and made a note of its number for you it is on account of the grownups and their ways', 'when you tell them that you have made a new friend they never ask you any questions about essential matters', 'they never say to you what does his voice sound like', 'what games does he love best', 'does he collect butterflies instead they demand how old is he', 'how many brothers has he', 'how much does he weigh', 'how much money does his father make only from these figures do they think they have learned anything about him', 'if you were to say to the grownups i saw a beautiful house made of rosy brick with geraniums in the windows and doves on the roof they would not be able to get any idea of that house at all', 'you would have to say to them i saw a house that cost   then they would exclaim oh what a pretty house that is just so you might say to them the proof that the little prince existed is that he was charming that he laughed and that he was looking for a sheep', 'if anybody wants a sheep that is a proof that he exists and what good would it do to tell them that', 'they would shrug their shoulders and treat you like a child', 'but if you said to them the planet he came from is asteroid b then they would be convinced and leave you in peace from their questions', 'they are like that', 'one must not hold it against them', 'children should always show great forbearance toward grownup people', 'but certainly for us who understand life figures are a matter of indifference', 'i should have liked to begin this story in the fashion of the fairytales', 'i should have like to say once upon a time there was a little prince who lived on a planet that was scarcely any bigger than himself and who had need of a sheep to those who understand life that would have given a much greater air of truth to my story', 'fori do not want any one to read my book carelessly', 'i have suffered too much grief in setting down these memories', 'six years have already passed since my friend went away from me with his sheep', 'if i try to describe him here it is to make sure that i shall not forget him', 'to forget a friend is sad', 'not every one has had a friend', 'and if i forget him i may become like the grownups who are no longer interested in anything but figures', 'it is for that purpose again that i have bought a box of paints and some pencils', 'it is hard to take up drawing again at my age when i have never made any pictures except those of the boa constrictor from the outside and the boa constrictor from the inside since i was six', 'i shall certainly try to make my portraits as true to life as possible', 'but i am not at all sure of success', 'one drawing goes along all right and another has no resemblance to its subject', 'i make some errors too in the little princes height in one place he is too tall and in another too short', 'and i feel some doubts about the colour of his costume', 'so i fumble along as best i can now good now bad and i hope generally fairto middling', 'in certain more important details i shall make mistakes also', 'but that is something that will not be my fault', 'my friend never explained anything to me', 'he thought perhaps that i was like himself', 'but i alas do not know how to see sheep through the walls of boxes', 'perhaps i am a little like the grownups', 'i have had to grow old', 'as each day passed i would learn in our talk something about the little princes planet his departure from it his journey', 'the information would come very slowly as it might chance to fall from his thoughts', 'it was in this way that i heard on the third day about the catastrophe of the baobabs', 'this time once more i had the sheep to thank for it', 'for the little prince asked me abruptly as if seized by a grave doubt it is true isnt it that sheep eat little bushes yes that is true ah', 'i am glad i did not understand why it was so important that sheep should eat little bushes', 'but the little prince added then it follows that they also eat baobabs i pointed out to the little prince that baobabs were not little bushes but on the contrary trees as big as castles and that even if he took a whole herd of elephants away with him the herd would not eat up one single baobab', 'the idea of the herd of elephants made the little prince laugh', 'we would have to put them one on top of the other he said', 'but he made a wise comment before they grow so big the baobabs start out by being little that is strictly correct i said', 'but why do you want the sheep to eat the little baobabs he answered me at once oh come come as if he were speaking of something that was self evident', 'and i was obliged to make a great mental effort to solve this problem without any assistance', 'indeed as i learned there were on the planet where the little prince lived as on all planets good plants and bad plants', 'in consequence there were good seeds from good plants and bad seeds from bad plants', 'but seeds are invisible', 'they sleep deep in the heart of the earths darkness until some one among them is seized with the desire to awaken', 'then this little seed will stretch itself and begin timidly at first to push a charming little sprig inoffensively upward toward the sun', 'if it is only a sprout of radish or the sprig of a rosebush one would let it grow wherever it might wish', 'but when it is a bad plant one must destroy it as soon as possible the very first instant that one recognises it', 'now there were some terrible seeds on the planet that was the home of the little prince and these were the seeds of the baobab', 'the soil of that planet was infested with them', 'a baobab is something you will never never be able to get rid of if you attend to it too late', 'it spreads over the entire planet', 'it bores clear through it with its roots', 'and if the planet is too small and the baobabs are too many they split it in pieces it is a question of discipline the little prince said to me later on', 'when youve finished your own toilet in the morning then it is time to attend to the toilet of your planet just so with the greatest care', 'you must see to it that you pull up regularly all the baobabs at the very first moment when they can be distinguished from the rosebushes which they resemble so closely in their earliest youth', 'it is very tedious work the little prince added but very easy and one day he said to me you ought to make a beautiful drawing so that the children where you live can see exactly how all this is', 'that would be very useful to them if they were to travel some day', 'sometimes he added there is no harm in putting off a piece of work until another day', 'but when it is a matter of baobabs that always means a catastrophe', 'i knew a planet that was inhabited by a lazy man', 'he neglected three little bushes', 'so as the little prince described it to me i have made a drawing of that planet', 'i do not much like to take the tone of a moralist', 'but the danger of the baobabs is so little understood and such considerable risks would be run by anyone who might get lost on an asteroid that for once i am breaking through my reserve', 'children i say plainly watch out for the baobabs my friends like myself have been skirting this danger for a long time without ever knowing it and so it is for them that i have worked so hard over this drawing', 'the lesson which i pass on by this means is worth all the trouble it has cost me', 'perhaps you will ask me why are there no other drawing in this book as magnificent and impressive as this drawing of the baobabs the reply is simple', 'i have tried', 'but with the others i have not been successful', 'when i made the drawing of the baobabs i was carried beyond myself by the inspiring force of urgent necessity', 'oh little prince', 'bit by bit i came to understand the secrets of your sad little life for a long time you had found your only entertainment in the quiet pleasure of looking at the sunset', 'i learned that new detail on the morning of the fourth day when you said to me i am very fond of sunsets', 'come let us go look at a sunset now but we must wait i said', 'wait', 'for what for the sunset', 'we must wait until it is time at first you seemed to be very much surprised', 'and then you laughed to yourself', 'you said to me i am always thinking that i am at home just so', 'everybody knows that when it is noon in the united states the sun is setting over france', 'if you could fly to france in one minute you could go straight into the sunset right from noon', 'unfortunately france is too far away for that', 'but on your tiny planet my little prince all you need do is move your chair a few steps', 'you can see the day end and the twilight falling whenever you like one day you said to me i saw the sunset fortyfour times and a little later you added you know one loves the sunset when one is so sad were you so sad then i asked on the day of the fortyfour sunsets but the little prince made no reply', 'on the fifth day again as always it was thanks to the sheep the secret of the little princes life was revealed to me', 'abruptly without anything to lead up to it and as if the question had been born of long and silent meditation on his problem he demanded a sheep if it eats little bushes does it eat flowers too a sheep i answered eats anything it finds in its reach even flowers that have thorns yes even flowers that have thorns then the thorns what use are they i did not know', 'at that moment i was very busy trying to unscrew a bolt that had got stuck in my engine', 'i was very much worried for it was becoming clear to me that the breakdown of my plane was extremely serious', 'and i had so little drinking water left that i had to fear for the worst', 'the thorns what use are they the little prince never let go of a question once he had asked it', 'as for me i was upset over that bolt', 'and i answered with the first thing that came into my head the thorns are of no use at all', 'flowers have thorns just for spite oh there was a moment of complete silence', 'then the little prince flashed back at me with a kind of resentfulness i dont believe you', 'flowers are weak creatures', 'they are naive', 'they reassure themselves as best they can', 'they believe that their thorns are terrible weapons i did not answer', 'at that instant i was saying to myself if this bolt still wont turn i am going to knock it out with the hammer again the little prince disturbed my thoughts', 'and you actually believe that the flowers oh no i cried', 'no no no', 'i dont believe anything', 'i answered you with the first thing that came into my head', 'dont you see i am very busy with matters of consequence he stared at me thunderstruck', 'matters of consequence he looked at me there with my hammer in my hand my fingers black with engine grease bending down over an object which seemed to him extremely ugly you talk just like the grownups that made me a little ashamed', 'but he went on relentlessly you mix everything up together you confuse everything he was really very angry', 'he tossed his golden curls in the breeze', 'i know a planet where there is a certain redfaced gentleman', 'he has never smelled a flower', 'he has never looked at a star', 'he has never loved any one', 'he has never done anything in his life but add up figures', 'and all day he says over and over just like you i am busy with matters of consequence and that makes him swell up with pride', 'but he is not a man he is a mushroom a what a mushroom the little prince was now white with rage', 'the flowers have been growing thorns for millions of years', 'for millions of years the sheep have been eating them just the same', 'and is it not a matter of consequence to try to understand why the flowers go to so much trouble to grow   i  thorns which are never of any use to them', 'is the warfare between the sheep and the flowers not important', 'is this not of more consequence than a fat redfaced gentlemans sums', 'and if i know i myself one flower which is unique in the world which grows nowhere but on my planet but which one little sheep can destroy in a single bite some morning without even noticing what he is doing oh', 'you think that is not important', 'his face turned from white to red as he continued if some one loves a flower of which just one single blossom grows in all the millions and millions of stars it is enough to make him happy just to look at the stars', 'he can say to himself somewhere my flower is there but if the sheep eats the flower in one moment all his stars will be darkened and you think that is not important he could not say anything more', 'his words were choked by sobbing', 'the night had fallen', 'i had let my tools drop from my hands', 'of what moment now was my hammer my bolt or thirst or death', 'on one star one planet my planet the earth there was a little prince to be comforted', 'i took him in my arms and rocked him', 'i said to him the flower that you love is not in danger', 'i will draw you a muzzle for your sheep', 'i will draw you a railing to put around your flower', 'i will i did not know what to say to him', 'i felt awkward and blundering', 'i did not know how i could reach him where i could overtake him and go on hand in hand with him once more', 'it is such a secret place the land of tears', 'i soon learned to know this flower better', 'on the little princes planet the flowers had always been very simple', 'they had only one ring of petals they took up no room at all they were a trouble to nobody', 'one morning they would appear in the grass and by night they would have faded peacefully away', 'but one day from a seed blown from no one knew where a new flower had come up and the little prince had watched very closely over this small sprout which was not like any other small sprouts on his planet', 'it might you see have been a new kind of baobab', 'the shrub soon stopped growing and began to get ready to produce a flower', 'the little prince who was present at the first appearance of a huge bud felt at once that some sort of miraculous apparition must emerge from it', 'but the flower was not satisfied to complete the preparations for her beauty in the shelter of her green chamber', 'she chose her colours with the greatest care', 'she adjusted her petals one by one', 'she did not wish to go out into the world all rumpled like the field poppies', 'it was only in the full radiance of her beauty that she wished to appear', 'oh yes', 'she was a coquettish creature', 'and her mysterious adornment lasted for days and days', 'then one morning exactly at sunrise she suddenly showed herself', 'and after working with all this painstaking precision she yawned and said ah', 'i am scarcely awake', 'i beg that you will excuse me', 'my petals are still all disarranged but the little prince could not restrain his admiration oh', 'how beautiful you are am i not the flower responded sweetly', 'and i was born at the same moment as the sun the little prince could guess easily enough that she was not any too modest but how moving and exciting she was', 'i think it is time for breakfast she added an instant later', 'if you would have the kindness to think of my needs and the little prince completely abashed went to look for a sprinkling can of fresh water', 'so he tended the flower', 'so too she began very quickly to torment him with her vanity which was if the truth be known a little difficult to deal with', 'one day for instance when she was speaking of her four thorns she said to the little prince let the tigers come with their claws there are no tigers on my planet the little prince objected', 'and anyway tigers do not eat weeds i am not a weed the flower replied sweetly', 'please excuse me i am not at all afraid of tigers she went on but i have a horror of drafts', 'i suppose you wouldnt screen for me', 'a horror of drafts that is bad luck for a plant remarked the little prince and added to himself this flower is a very complex creature at night i want you to put me under a glass globe', 'it is very cold where you live', 'in the place i came from but she interrupted herself at that point', 'she had come in the form of a seed', 'she could not have known anything of any other worlds', 'embarrassed over having let herself be caught on the verge of such an untruth she coughed two or three times in order to put the little prince in the wrong', 'the screen i was just going to look for it when you spoke to me then she forced her cough a little more so that he should suffer from remorse just the same', 'so the little prince in spite of all the good will that was inseparable from his love had soon come to doubt her', 'he had taken seriously words which were without importance and it made him very unhappy', 'i ought not to have listened to her he confided to me one day', 'one never ought to listen to the flowers', 'one should simply look at them and breathe their fragrance', 'mine perfumed all my planet', 'but i did not know how to take pleasure in all her grace', 'this tale of claws which disturbed me so much should only have filled my heart with tenderness and pity and he continued his confidences the fact is that i did not know how to understand anything', 'i ought to have judged by deeds and not by words', 'she cast her fragrance and her radiance over me', 'i ought never to have run away from her', 'i ought to have guessed all the affection that lay behind her poor little stratagems', 'flowers are so inconsistent', 'but i was too young to know how to love her i believe that for his escape he took advantage of the migration of a flock of wild birds', 'on the morning of his departure he put his planet in perfect order', 'he carefully cleaned out his active volcanoes', 'he possessed two active volcanoes and they were very convenient for heating his breakfast in the morning', 'he also had one volcano that was extinct', 'but as he said one never knows so he cleaned out the extinct volcano too', 'if they are well cleaned out volcanoes burn slowly and steadily without any eruptions', 'volcanic eruptions are like fires in a chimney', 'on our earth we are obviously much too small to clean out our volcanoes', 'that is why they bring no end of trouble upon us', 'the little prince also pulled up with a certain sense of dejection the last little shoots of the baobabs', 'he believed that he would never want to return', 'but on this last morning all these familiar tasks seemed very precious to him', 'and when he watered the flower for the last time and prepared to place her under the shelter of her glass globe he realised that he was very close to tears', 'goodbye he said to the flower', 'but she made no answer', 'goodbye he said again', 'the flower coughed', 'but it was not because she had a cold', 'i have been silly she said to him at last', 'i ask your forgiveness', 'try to be happy he was surprised by this absence of reproaches', 'he stood there all bewildered the glass globe held arrested in midair', 'he did not understand this quiet sweetness', 'of course i love you the flower said to him', 'it is my fault that you have not known it all the while', 'that is of no importance', 'but you you have been just as foolish as i', 'try to be happy let the glass globe be', 'i dont want it any more but the wind my cold is not so bad as all that the cool night air will do me good', 'i am a flower but the animals well i must endure the presence of two or three caterpillars if i wish to become acquainted with the butterflies', 'it seems that they are very beautiful', 'and if not the butterflies and the caterpillars who will call upon me', 'you will be far away as for the large animals i am not at all afraid of any of them', 'i have my claws and naively she showed her four thorns', 'then she added dont linger like this', 'you have decided to go away', 'now go for she did not want him to see her crying', 'she was such a proud flower', 'he found himself in the neighbourhood of the asteroids      and ', 'he began therefore by visiting them in order to add to his knowledge', 'the first of them was inhabited by a king', 'clad in royal purple and ermine he was seated upon a throne which was at the same time both simple and majestic', 'ah', 'here is a subject exclaimed the king when he saw the little prince coming', 'and the little prince asked himself how could he recognise me when he had never seen me before he did not know how the world is simplified for kings', 'to them all men are subjects', 'approach so that i may see you better said the king who felt consumingly proud of being at last a king over somebody', 'the little prince looked everywhere to find a place to sit down but the entire planet was crammed and obstructed by the kings magnificent ermine robe', 'so he remained standing upright and since he was tired he yawned', 'it is contrary to etiquette to yawn in the presence of a king the monarch said to him', 'i forbid you to do so i cant help it', 'i cant stop myself replied the little prince thoroughly embarrassed', 'i have come on a long journey and i have had no sleep ah then the king said', 'i order you to yawn', 'it is years since i have seen anyone yawning', 'yawns to me are objects of curiosity', 'come now', 'yawn again', 'it is an order that frightens me', 'i cannot any more murmured the little prince now completely abashed', 'hum', 'hum replied the king', 'then i', 'i order you sometimes to yawn and sometimes to he sputtered a little and seemed vexed', 'for what the king fundamentally insisted upon was that his authority should be respected', 'he tolerated no disobedience', 'he was an absolute monarch', 'but because he was a very good man he made his orders reasonable', 'if i ordered a general he would say by way of example if i ordered a general to change himself into a sea bird and if the general did not obey me that would not be the fault of the general', 'it would be my fault may i sit down came now a timid inquiry from the little prince', 'i order you to do so the king answered him and majestically gathered in a fold of his ermine mantle', 'but the little prince was wondering', 'the planet was tiny', 'over what could this king really rule', 'sire he said to him i beg that you will excuse my asking you a question i order you to ask me a question the king hastened to assure him', 'sire over what do you rule over everything said the king with magnificent simplicity', 'over everything the king made a gesture which took in his planet the other planets and all the stars', 'over all that asked the little prince', 'over all that the king answered', 'for his rule was not only absolute it was also universal', 'and the stars obey you certainly they do the king said', 'they obey instantly', 'i do not permit insubordination such power was a thing for the little prince to marvel at', 'if he had been master of such complete authority he would have been able to watch the sunset not fortyfour times in one day but seventytwo or even a hundred or even two hundred times with out ever having to move his chair', 'and because he felt a bit sad as he remembered his little planet which he had forsaken he plucked up his courage to ask the king a favour i should like to see a sunset do me that kindness order the sun to set if i ordered a general to fly from one flower to another like a butterfly or to write a tragic drama or to change himself into a sea bird and if the general did not carry out the order that he had received which one of us would be in the wrong the king demanded', 'the general or myself you said the little prince firmly', 'exactly', 'one much require from each one the duty which each one can perform the king went on', 'accepted authority rests first of all on reason', 'if you ordered your people to go and throw themselves into the sea they would rise up in revolution', 'i have the right to require obedience because my orders are reasonable then my sunset the little prince reminded him for he never forgot a question once he had asked it', 'you shall have your sunset', 'i shall command it', 'but according to my science of government i shall wait until conditions are favourable when will that be inquired the little prince', 'hum', 'hum replied the king and before saying anything else he consulted a bulky almanac', 'hum', 'hum', 'that will be about about that will be this evening about twenty minutes to eight', 'and you will see how well i am obeyed the little prince yawned', 'he was regretting his lost sunset', 'and then too he was already beginning to be a little bored', 'i have nothing more to do here he said to the king', 'so i shall set out on my way again do not go said the king who was very proud of having a subject', 'do not go', 'i will make you a minister minister of what minster ofof justice but there is nobody here to judge we do not know that the king said to him', 'i have not yet made a complete tour of my kingdom', 'i am very old', 'there is no room here for a carriage', 'and it tires me to walk oh but i have looked already said the little prince turning around to give one more glance to the other side of the planet', 'on that side as on this there was nobody at all then you shall judge yourself the king answered that is the most difficult thing of all', 'it is much more difficult to j udge oneself than to j udge others', 'if you succeed in j udging yourself rightly then you are indeed a man of true wisdom yes said the little prince but i can judge myself anywhere', 'i do not need to live on this planet', 'hum', 'hum said the king', 'i have good reason to believe that somewhere on my planet there is an old rat', 'i hear him at night', 'you can judge this old rat', 'from time to time you will condemn him to death', 'thus his life will depend on your j ustice', 'but you will pardon him on each occasion for he must be treated thriftily', 'he is the only one we have i replied the little prince do not like to condemn anyone to death', 'and now i think i will go on my way no said the king', 'but the little prince having now completed his preparations for departure had no wish to grieve the old monarch', 'if your majesty wishes to be promptly obeyed he said he should be able to give me a reasonable order', 'he should be able for example to order me to be gone by the end of one minute', 'it seems to me that conditions are favourable as the king made no answer the little prince hesitated a moment', 'then with a sigh he took his leave', 'i made you my ambassador the king called out hastily', 'he had a magnificent air of authority', 'the grownups are very strange the little prince said to himself as he continued on his journey', 'the second planet was inhabited by a conceited man', 'ah', 'ah', 'i am about to receive a visit from an admirer he exclaimed from afar when he first saw the little prince coming', 'for to conceited men all other men are admirers', 'good morning said the little prince', 'that is a queer hat you are wearing it is a hat for salutes the conceited man replied', 'it is to raise in salute when people acclaim me', 'unfortunately nobody at all ever passes this way yes said the little prince who did not understand what the conceited man was talking about', 'clap your hands one against the other the conceited man now directed him', 'the little prince clapped his hands', 'the conceited man raised his hat in a modest salute', 'this is more entertaining than the visit to the king the little prince said to himself', 'and he began again to clap his hands one against the other', 'the conceited man against raised his hat in salute', 'after five minutes of this exercise the little prince grew tired of the games monotony', 'and what should one do to make the hat come down he asked', 'but the conceited man did not hear him', 'conceited people never hear anything but praise', 'do you really admire me very much he demanded of the little prince', 'what does that mean admire to admire means that you regard me as the handsomest the bestdressed the richest and the most intelligent man on this planet but you are the only man on your planet do me this kindness', 'admire me just the same i admire you said the little prince shrugging his shoulders slightly but what is there in that to interest you so much and the little prince went away', 'the grownups are certainly very odd he said to himself as he continued on his journey', 'the next planet was inhabited by a tippler', 'this was a very short visit but it plunged the little prince into deep dejection', 'what are you doing there he said to the tippler whom he found settled down in silence before a collection of empty bottles and also a collection of full bottles', 'i am drinking replied the tippler with a lugubrious air', 'why are you drinking demanded the little prince', 'so that i may forget replied the tippler', 'forget what inquired the little prince who already was sorry for him', 'forget that i am ashamed the tippler confessed hanging his head', 'ashamed of what insisted the little prince who wanted to help him', 'ashamed of drinking the tippler brought his speech to an end and shut himself up in an impregnable silence', 'and the little prince went away puzzled', 'the grownups are certainly very very odd he said to himself as he continued on his journey', 'the fourth planet belonged to a businessman', 'this man was so much occupied that he did not even raise his head at the little princes arrival', 'good morning the little prince said to him', 'your cigarette has gone out three and two make five', 'five and seven make twelve', 'twelve and three make fifteen', 'good morning', 'fifteen and seven make twentytwo', 'twentytwo and six make twentyeight', 'i havent time to light it again', 'twentysix and five make thirtyone', 'phew ', 'then that makes fivehundred andonemillion sixhundredtwentytwothousand sevenhundredthirtyone five hundred million what asked the little prince', 'eh', 'are you still there', 'fivehundredand one million i cant stop', 'i have so much to do', 'i am concerned with matters of consequence', 'i dont amuse myself with balderdash', 'two and five make seven fivehundredandone million what repeated the little prince who never in his life had let go of a question once he had asked it', 'the businessman raised his head', 'during the fiftyfour years that i have inhabited this planet i have been disturbed only three times', 'the first time was twentytwo years ago when some giddy goose fell from goodness knows where', 'he made the most frightful noise that resounded all over the place and i made four mistakes in my addition', 'the second time eleven years ago i was disturbed by an attack of rheumatism', 'i dont get enough exercise', 'i have no time for loafing', 'the third time well this is it', 'i was saying then five hundredandone millions millions of what the businessman suddenly realised that there was no hope of being left in peace until he answered this question', 'millions of those little objects he said which one sometimes sees in the sky flies oh no', 'little glittering objects bees oh no', 'little golden objects that set lazy men to idle dreaming', 'as for me i am concerned with matters of consequence', 'there is no time for idle dreaming in my life ah', 'you mean the stars yes thats it', 'the stars and what do you do with fivehundred millions of stars fivehundredandone million sixhundredtwentytwo thousand sevenhundredthirtyone', 'i am concerned with matters of consequence i am accurate and what do you do with these stars what do i do with them yes nothing', 'i own them you own the stars yes but i have already seen a king who kings do not own they reign over', 'it is a very different matter and what good does it do you to own the stars it does me the good of making me rich and what good does it do you to be rich it makes it possible for me to buy more stars if any are ever discovered this man the little prince said to himself reasons a little like my poor tippler nevertheless he still had some more questions', 'how is it possible for one to own the stars to whom do they belong the businessman retorted peevishly', 'i dont know', 'to nobody then they belong to me because i was the first person to think of it is that all that is necessary certainly', 'when you find a diamond that belongs to nobody it is yours', 'when you discover an island that belongs to nobody it is yours', 'when you get an idea before any one else you take out a patent on it it is yours', 'so with me i own the stars because nobody else before me ever thought of owning them yes that is true said the little prince', 'and what do you do with them i administer them replied the businessman', 'i count them and recount them', 'it is difficult', 'but i am a man who is naturally interested in matters of consequence the little prince was still not satisfied', 'if i owned a silk scarf he said i could put it around my neck and take it away with me', 'if i owned a flower i could pluck that flower and take it away with me', 'but you cannot pluck the stars from heaven no', 'but i can put them in the bank whatever does that mean that means that i write the number of my stars on a little paper', 'and then i put this paper in a drawer and lock it with a key and that is all that is enough said the businessman', 'it is entertaining thought the little prince', 'it is rather poetic', 'but it is of no great consequence on matters of consequence the little prince had ideas which were very different from those of the grownups', 'i myself own a flower he continued his conversation with the businessman which i water every day', 'i own three volcanoes which i clean out every week for i also clean out the one that is extinct one never knows', 'it is of some use to my volcanoes and it is of some use to my flower that i own them', 'but you are of no use to the stars the businessman opened his mouth but he found nothing to say in answer', 'and the little prince went away', 'the grownups are certainly altogether extraordinary he said simply talking to himself as he continued on his journey', 'the fifth planet was very strange', 'it was the smallest of all', 'there was just enough room on it for a street lamp and a lamplighter', 'the little prince was not able to reach any explanation of the use of a street lamp and a lamplighter somewhere in the heavens on a planet which had no people and not one house', 'but he said to himself nevertheless it may well be that this man is absurd', 'but he is not so absurd as the king the conceited man the businessman and the tippler', 'for at least his work has some meaning', 'when he lights his street lamp it is as if he brought one more star to life or one flower', 'when he puts out his lamp he sends the flower or the star to sleep', 'that is a beautiful occupation', 'and since it is beautiful it is truly useful when he arrived on the planet he respectfully saluted the lamplighter', 'good morning', 'why have you just put out your lamp those are the orders replied the lamplighter', 'good morning what are the orders the orders are that i put out my lamp', 'good evening and he lighted his lamp again', 'but why have you just lighted it again those are the orders replied the lamplighter', 'i do not understand said the little prince', 'there is nothing to understand said the lamplighter', 'orders are orders', 'good morning and he put out his lamp', 'then he mopped his forehead with a handkerchief decorated with red squares', 'i follow a terrible profession', 'in the old days it was reasonable', 'i put the lamp out in the morning and in the evening i lighted it again', 'i had the rest of the day for relaxation and the rest of the night for sleep and the orders have been changed since that time the orders have not been changed said the lamplighter', 'that is the tragedy', 'from year to year the planet has turned more rapidly and the orders have not been changed then what asked the little prince', 'then the planet now makes a complete turn every minute and i no longer have a single second for repose', 'once every minute i have to light my lamp and put it out that is very funny', 'a day lasts only one minute here where you live it is not funny at all said the lamplighter', 'while we have been talking together a month has gone by a month yes a month', 'thirty minutes', 'thirty days', 'good evening and he lighted his lamp again', 'as the little prince watched him he felt that he loved this lamplighter who was so faithful to his orders', 'he remembered the sunsets which he himself had gone to seek in other days merely by pulling up his chair and he wanted to help his friend', 'you know he said i can tell you a way you can rest whenever you want to i always want to rest said the lamplighter', 'for it is possible for a man to be faithful and lazy at the same time', 'the little prince went on with his explanation your planet is so small that three strides will take you all the way around it', 'to be always in the sunshine you need only walk along rather slowly', 'when you want to rest you will walk and the day will last as long as you like that doesnt do me much good said the lamplighter', 'the one thing i love in life is to sleep then youre unlucky said the little prince', 'i am unlucky said the lamplighter', 'good morning and he put out his lamp', 'that man said the little prince to himself as he continued farther on his journey that man would be scorned by all the others by the king by the conceited man by the tippler by the businessman', 'nevertheless he is the only one of them all who does not seem to me ridiculous', 'perhaps that is because he is thinking of something else besides himself he breathed a sigh of regret and said to himself again that man is the only one of them all whom i could have made my friend', 'but his planet is indeed too small', 'there is no room on it for two people what the little prince did not dare confess was that he was sorry most of all to leave this planet because it was blest every day with  sunsets', 'the sixth planet was ten times larger than the last one', 'it was inhabited by an old gentleman who wrote voluminous books', 'oh look', 'here is an explorer he exclaimed to himself when he saw the little prince coming', 'the little prince sat down on the table and panted a little', 'he had already travelled so much and so far', 'where do you come from the old gentleman said to him', 'what is that big book said the little prince', 'what are you doing i am a geographer the old gentleman said to him', 'what is a geographer asked the little prince', 'a geographer is a scholar who knows the location of all the seas rivers towns mountains and deserts that is very interesting said the little prince', 'here at last is a man who has a real profession and he cast a look around him at the planet of the geographer', 'it was the most magnificent and stately planet that he had ever seen', 'your planet is very beautiful he said', 'has it any oceans i couldnt tell you said the geographer', 'ah the little prince was disappointed', 'has it any mountains i couldnt tell you said the geographer', 'and towns and rivers and deserts i couldnt tell you that either but you are a geographer exactly the geographer said', 'but i am not an explorer', 'i havent a single explorer on my planet', 'it is not the geographer who goes out to count the towns the rivers the mountains the seas the oceans and the deserts', 'the geographer is much too important to go loafing about', 'he does not leave his desk', 'but he receives the explorers in his study', 'he asks them questions and he notes down what they recall of their travels', 'and if the recollections of any one among them seem interesting to him the geographer orders an inquiry into that explorers moral character why is that because an explorer who told lies would bring disaster on the books of the geographer', 'so would an explorer who drank too much why is that asked the little prince', 'because intoxicated men see double', 'then the geographer would note down two mountains in a place where there was only one i know some one said the little prince who would make a bad explorer that is possible', 'then when the moral character of the explorer is shown to be good an inquiry is ordered into his discovery one goes to see it no', 'that would be too complicated', 'but one requires the explorer to furnish proofs', 'for example if the discovery in question is that of a large mountain one requires that large stones be brought back from it the geographer was suddenly stirred to excitement', 'but you come from far away', 'you are an explorer', 'you shall describe your planet to me and having opened his big register the geographer sharpened his pencil', 'the recitals of explorers are put down first in pencil', 'one waits until the explorer has furnished proofs before putting them down in ink', 'well said the geographer expectantly', 'oh where i live said the little prince it is not very interesting', 'it is all so small', 'i have three volcanoes', 'two volcanoes are active and the other is extinct', 'but one never knows one never knows said the geographer', 'i have also a flower we do not record flowers said the geographer', 'why is that', 'the flower is the most beautiful thing on my planet we do not record them said the geographer because they are ephemeral what does that mean ephemeral geographies said the geographer are the books which of all books are most concerned with matters of consequence', 'they never become oldfashioned', 'it is very rarely that a mountain changes its position', 'it is very rarely that an ocean empties itself of its waters', 'we write of eternal things but extinct volcanoes may come to life again the little prince interrupted', 'what does that mean ephemeral whether volcanoes are extinct or alive it comes to the same thing for us said the geographer', 'the thing that matters to us is the mountain', 'it does not change but what does that mean ephemeral repeated the little prince who never in his life had let go of a question once he had asked it', 'it means which is in danger of speedy disappearance  is my flower in danger of speedy disappearance certainly it is my flower is ephemeral the little prince said to himself and she has only four thorns to defend herself against the world', 'and i have left her on my planet all alone that was his first moment of regret', 'but he took courage once more', 'what place would you advise me to visit now he asked', 'the planet earth replied the geographer', 'it has a good reputation and the little prince went away thinking of his flower', 'so then the seventh planet was the earth', 'the earth is not just an ordinary planet', 'one can count there  kings not forgetting to be sure the negro kings among them  geographers  businessmen  tipplers  conceited men that is to say about  grownups', 'to give you an idea of the size of the earth i will tell you that before the invention of electricity it was necessary to maintain over the whole of the six continents a veritable army of  lamplighters for the street lamps', 'seen from a slight distance that would make a splendid spectacle', 'the movements of this army would be regulated like those of the ballet in the opera', 'first would come the turn of the lamplighters of new zealand and australia', 'having set their lamps alight these would go off to sleep', 'next the lamplighters of china and siberia would enter for their steps in the dance and then they too would be waved back into the wings', 'after that would come the turn of the lamplighters of russia and the indies then those of africa and europe then those of south america then those of north america', 'and never would they make a mistake in the order of their entry upon the stage', 'it would be magnificent', 'only the man who was in charge of the single lamp at the north pole and his colleague who was responsible for the single lamp at the south pole only these two would live free from toil and care they would be busy twice a year', 'when one wishes to play the wit he sometimes wanders a little from the truth', 'i have not been altogether honest in what i have told you about the lamplighters', 'and i realise that i run the risk of giving a false idea of our planet to those who do not know it', 'men occupy a very small place upon the earth', 'if the two billion inhabitants who people its surface were all to stand upright and somewhat crowded together as they do for some big public assembly they could easily be put into one public square twenty miles long and twenty miles wide', 'all humanity could be piled up on a small pacific islet', 'the grownups to be sure will not believe you when you tell them that', 'they imagine that they fill a great deal of space', 'they fancy themselves as important as the baobabs', 'you should advise them then to make their own calculations', 'they adore figures and that will please them', 'but do not waste your time on this extra task', 'it is unnecessary', 'you have i know confidence in me', 'when the little prince arrived on the earth he was very much surprised not to see any people', 'he was beginning to be afraid he had come to the wrong planet when a coil of gold the colour of the moonlight flashed across the sand', 'good evening said the little prince courteously', 'good evening said the snake', 'what planet is this on which i have come down asked the little prince', 'this is the earth this is africa the snake answered', 'ah', 'then there are no people on the earth this is the desert', 'there are no people in the desert', 'the earth is large said the snake', 'the little prince sat down on a stone and raised his eyes toward the sky', 'i wonder he said whether the stars are set alight in heaven so that one day each one of us may find his own again look at my planet', 'it is right there above us', 'but how far away it is it is beautiful the snake said', 'what has brought you here i have been having some trouble with a flower said the little prince', 'ah said the snake', 'and they were both silent', 'where are the men the little prince at last took up the conversation again', 'it is a little lonely in the desert it is also lonely among men the snake said', 'the little prince gazed at him for a long time', 'you are a funny animal he said at last', 'you are no thicker than a finger but i am more powerful than the finger of a king said the snake', 'the little prince smiled', 'you are not very powerful', 'you havent even any feet', 'you cannot even travel i can carry you farther than any ship could take you said the snake', 'he twined himself around the little princes ankle like a golden bracelet', 'whomever i touch i send back to the earth from whence he came the snake spoke again', 'but you are innocent and true and you come from a star the little prince made no reply', 'you move me to pity you are so weak on this earth made of granite the snake said', 'i can help you some day if you grow too homesick for your own planet', 'i can oh', 'i understand you very well said the little prince', 'but why do you always speak in riddles i solve them all said the snake', 'and they were both silent', 'the little prince crossed the desert and met with only one flower', 'it was a flower with three petals a flower of no account at all', 'good morning said the little prince', 'good morning said the flower', 'where are the men the little prince asked politely', 'the flower had once seen a caravan passing', 'men she echoed', 'i think there are six or seven of them in existence', 'i saw them several years ago', 'but one never knows where to find them', 'the wind blows them away', 'they have no roots and that makes their life very difficult goodbye said the little prince', 'goodbye said the flower', 'after that the little prince climbed a high mountain', 'the only mountains he had ever known were the three volcanoes which came up to his knees', 'and he used the extinct volcano as a footstool', 'from a mountain as high as this one he said to himself i shall be able to see the whole planet at one glance and all the people but he saw nothing save peaks of rock that were sharpened like needles', 'good morning he said courteously', 'good morninggood morninggood morning answered the echo', 'who are you said the little prince', 'who are youwho are youwho are you answered the echo', 'be my friends', 'i am all alone he said', 'i am all aloneall alone', 'all alone answered the echo', 'what a queer planet he thought', 'it is altogether dry and altogether pointed and altogether harsh and forbidding', 'and the people have no imagination', 'they repeat whatever one says to them on my planet i had a flower she always was the first to speak but it happened that after walking for a long time through sand and rocks and snow the little prince at last came upon a road', 'and all roads lead to the abodes of men', 'good morning he said', 'he was standing before a garden all abloom with roses', 'good morning said the roses', 'the little prince gazed at them', 'they all looked like his flower', 'who are you he demanded thunderstruck', 'we are roses the roses said', 'and he was overcome with sadness', 'his flower had told him that she was the only one of her kind in all the universe', 'and here were five thousand of them all alike in one single garden', 'she would be very much annoyed he said to himself if she should see that she would cough most dreadfully and she would pretend that she was dying to avoid being laughed at', 'and i should be obliged to pretend that i was nursing her back to life for if i did not do that to humble myself also she would really allow herself to die then he went on with his reflections i thought that i was rich with a flower that was unique in all the world and all i had was a common rose', 'a common rose and three volcanoes that come up to my knees  and one of them perhaps extinct forever that doesnt make me a very great prince and he lay down in the grass and cried', 'it was then that the fox appeared', 'good morning said the fox', 'good morning the little prince responded politely although when he turned around he saw nothing', 'i am right here the voice said under the apple tree  who are you asked the little prince and added you are very pretty to look at i am a fox said the fox', 'come and play with me proposed the little prince', 'i am so unhappy i cannot play with you the fox said', 'i am not tamed ah', 'please excuse me said the little prince', 'but after some thought he added what does that mean tame you do not live here said the fox', 'what is it that you are looking for i am looking for men said the little prince', 'what does that mean tame men said the fox', 'they have guns and they hunt', 'it is very disturbing', 'they also raise chickens', 'these are their only interests', 'are you looking for chickens no said the little prince', 'i am looking for friends', 'what does that mean tame it is an act too often neglected said the fox', 'it means to establish ties to establish ties just that said the fox', 'to me you are still nothing more than a little boy who is just like a hundred thousand other little boys', 'and i have no need of you', 'and you on your part have no need of me', 'to you i am nothing more than a fox like a hundred thousand other foxes', 'but if you tame me then we shall need each other', 'to me you will be unique in all the world', 'to you i shall be unique in all the world i am beginning to understand said the little prince', 'there is a flower', 'i think that she has tamed me it is possible said the fox', 'on the earth one sees all sorts of things oh but this is not on the earth said the little prince', 'the fox seemed perplexed and very curious', 'on another planet yes are there hunters on this planet no ah that is interesting', 'are there chickens no nothing is perfect sighed the fox', 'but he came back to his idea', 'my life is very monotonous the fox said', 'i hunt chickens men hunt me', 'all the chickens are just alike and all the men are just alike', 'and in consequence i am a little bored', 'but if you tame me it will be as if the sun came to shine on my life', 'i shall know the sound of a step that will be different from all the others', 'other steps send me hurrying back underneath the ground', 'yours will call me like music out of my burrow', 'and then look you see the grainfields down yonder', 'i do not eat bread', 'wheat is of no use to me', 'the wheat fields have nothing to say to me', 'and that is sad', 'but you have hair that is the colour of gold', 'think how wonderful that will be when you have tamed me', 'the grain which is also golden will bring me back the thought of you', 'and i shall love to listen to the wind in the wheat the fox gazed at the little prince for a long time', 'please tame me he said', 'i want to very much the little prince replied', 'but i have not much time', 'i have friends to discover and a great many things to understand one only understands the things that one tames said the fox', 'men have no more time to understand anything', 'they buy things all readymade at the shops', 'but there is no shop anywhere where one can buy friendship and so men have no friends any more', 'if you want a friend tame me what must i do to tame you asked the little prince', 'you must be very patient replied the fox', 'first you will sit down at a little distance from me like that in the grass', 'i shall look at you out of the corner of my eye and you will say nothing', 'words are the source of misunderstandings', 'but you will sit a little closer to me every day the next day the little prince came back', 'it would have been better to come back at the same hour said the fox', 'if for example you come at four oclock in the afternoon then at three oclock i shall begin to be happy', 'i shall feel happier and happier as the hour advances', 'at four oclock i shall already be worrying and jumping about', 'i shall show you how happy i am', 'but if you come at just any time i shall never know at what hour my heart is to be ready to greet you one must observe the proper rites what is a rite asked the little prince', 'those also are actions too often neglected said the fox', 'they are what make one day different from other days one hour from other hours', 'there is a rite for example among my hunters', 'every thursday they dance with the village girls', 'so thursday is a wonderful day for me', 'i can take a walk as far as the vineyards', 'but if the hunters danced at just any time every day would be like every other day and i should never have any vacation at all so the little prince tamed the fox', 'and when the hour of his departure drew near ah said the fox i shall cry it is your own fault said the little prince', 'i never wished you any sort of harm but you wanted me to tame you yes that is so said the fox', 'but now you are going to cry said the little prince', 'yes that is so said the fox', 'then it has done you no good at all it has done me good said the fox because of the colour of the wheat fields and then he added go and look again at the roses', 'you will understand now that yours is unique in all the world', 'then come back to say goodbye to me and i will make you a present of a secret the little prince went away to look again at the roses', 'you are not at all like my rose he said', 'as yet you are nothing', 'no one has tamed you and you have tamed no one', 'you are like my fox when i first knew him', 'he was only a fox like a hundred thousand other foxes', 'but i have made him my friend and now he is unique in all the world and the roses were very much embarrassed', 'you are beautiful but you are empty he went on', 'one could not die for you', 'to be sure an ordinary passerby would think that my rose looked just like you the rose that belongs to me', 'but in herself alone she is more important than all the hundreds of you other roses because it is she that i have watered because it is she that i have put under the glass globe because it is she that i have sheltered behind the screen because it is for her that i have killed the caterpillars except the two or three that we saved to become butterflies because it is she that i have listened to when she grumbled or boasted or even sometimes when she said nothing', 'because she is my rose', 'and he went back to meet the fox', 'goodbye he said', 'goodbye said the fox', 'and now here is my secret a very simple secret it is only with the heart that one can see rightly what is essential is invisible to the eye what is essential is invisible to the eye the little prince repeated so that he would be sure to remember', 'it is the time you have wasted for your rose that makes your rose so important it is the time i have wasted for my rose said the little prince so that he would be sure to remember', 'men have forgotten this truth said the fox', 'but you must not forget it', 'you become responsible forever for what you have tamed', 'you are responsible for your rose i am responsible for my rose the little prince repeated so that he would be sure to remember', 'good morning said the little prince', 'good morning said the railway switchman', 'what do you do here the little prince asked', 'i sort out travellers in bundles of a thousand said the switchman', 'i send off the trains that carry them now to the right now to the left and a brilliantly lighted express train shook the switchmans cabin as it rushed by with a roar like thunder', 'they are in a great hurry said the little prince', 'what are they looking for not even the locomotive engineer knows that said the switchman', 'and a second brilliantly lighted express thundered by in the opposite direction', 'are they coming back already demanded the little prince', 'these are not the same ones said the switchman', 'it is an exchange were they not satisfied where they were asked the little prince', 'no one is ever satisfied where he is said the switchman', 'and they heard the roaring thunder of a third brilliantly lighted express', 'are they pursuing the first travellers demanded the little prince', 'they are pursuing nothing at all said the switchman', 'they are asleep in there or if they are not asleep they are yawning', 'only the children are flattening their noses against the windowpanes only the children know what they are looking for said the little prince', 'they waste their time over a rag doll and it becomes very important to them and if anybody takes it away from them they cry they are lucky the switchman said', 'good morning said the little prince', 'good morning said the merchant', 'this was a merchant who sold pills that had been invented to quench thirst', 'you need only swallow one pill a week and you would feel no need of anything to drink', 'why are you selling those asked the little prince', 'because they save a tremendous amount of time said the merchant', 'computations have been made by experts', 'with these pills you save fiftythree minutes in every week and what do i do with those fiftythree minutes anything you like as for me said the little prince to himself if i had fiftythree minutes to spend as i liked i should walk at my leisure toward a spring of fresh water it was now the eighth day since i had had my accident in the desert and i had listened to the story of the merchant as i was drinking the last drop of my water supply', 'ah i said to the little prince these memories of yours are very charming but i have not yet succeeded in repairing my plane i have nothing more to drink and i too should be very happy if i could walk at my leisure toward a spring of fresh water my friend the fox the little prince said to me', 'my dear little man this is no longer a matter that has anything to do with the fox why not because i am about to die of thirst he did not follow my reasoning and he answered me it is a good thing to have had a friend even if one is about to die', 'i for instance am very glad to have had a fox as a friend he has no way of guessing the danger i said to myself', 'he has never been either hungry or thirsty', 'a little sunshine is all he needs but he looked at me steadily and replied to my thought i am thirsty too', 'let us look for a well i made a gesture of weariness', 'it is absurd to look for a well at random in the immensity of the desert', 'but nevertheless we started walking', 'when we had trudged along for several hours in silence the darkness fell and the stars began to come out', 'thirst had made me a little feverish and i looked at them as if i were in a dream', 'the little princes last words came reeling back into my memory then you are thirsty too i demanded', 'but he did not reply to my question', 'he merely said to me water may also be good for the heart i did not understand this answer but i said nothing', 'i knew very well that it was impossible to crossexamine him', 'he was tired', 'he sat down', 'i sat down beside him', 'and after a little silence he spoke again the stars are beautiful because of a flower that cannot be seen i replied yes that is so and without saying anything more i looked across the ridges of sand that were stretched out before us in the moonlight', 'the desert is beautiful the little prince added', 'and that was true', 'i have always loved the desert', 'one sits down on a desert sand dune sees nothing hears nothing', 'yet through the silence something throbs and gleams what makes the desert beautiful said the little prince is that somewhere it hides a well i was astonished by a sudden understanding of that mysterious radiation of the sands', 'when i was a little boy i lived in an old house and legend told us that a treasure was buried there', 'to be sure no one had ever known how to find it perhaps no one had ever even looked for it', 'but it cast an enchantment over that house', 'my home was hiding a secret in the depths of its heart yes i said to the little prince', 'the house the stars the desert  what gives them their beauty is something that is invisible i am glad he said that you agree with my fox as the little prince dropped off to sleep i took him in my arms and set out walking once more', 'i felt deeply moved and stirred', 'it seemed to me that i was carrying a very fragile treasure', 'it seemed to me even that there was nothing more fragile on all earth', 'in the moonlight i looked at his pale forehead his closed eyes his locks of hair that trembled in the wind and i said to myself what i see here is nothing but a shell', 'what is most important is invisible as his lips opened slightly with the suspicious of a halfsmile i said to myself again what moves me so deeply about this little prince who is sleeping here is his loyalty to a flower  the image of a rose that shines through his whole being like the flame of a lamp even when he is asleep and i felt him to be more fragile still', 'i felt the need of protecting him as if he himself were a flame that might be extinguished by a little puff of wind and as i walked on so i found the well at daybreak', 'men said the little prince set out on their way in express trains but they do not know what they are looking for', 'then they rush about and get excited and turn round and round and he added it is not worth the trouble the well that we had come to was not like the wells of the sahara', 'the wells of the sahara are mere holes dug in the sand', 'this one was like a well in a village', 'but there was no village here and i thought i must be dreaming it is strange i said to the little prince', 'everything is ready for use the pulley the bucket the rope he laughed touched the rope and set the pulley to working', 'and the pulley moaned like an old weathervane which the wind has long since forgotten', 'do you hear said the little prince', 'we have wakened the well and it is singing i did not want him to tire himself with the rope', 'leave it to me i said', 'it is too heavy for you i hoisted the bucket slowly to the edge of the well and set it there happy tired as i was over my achievement', 'the song of the pulley was still in my ears and i could see the sunlight shimmer in the still trembling water', 'i am thirsty for this water said the little prince', 'give me some of it to drink and i understood what he had been looking for', 'i raised the bucket to his lips', 'he drank his eyes closed', 'it was as sweet as some special festival treat', 'this water was indeed a different thing from ordinary nourishment', 'its sweetness was born of the walk under the stars the song of the pulley the effort of my arms', 'it was good for the heart like a present', 'when i was a little boy the lights of the christmas tree the music of the midnight mass the tenderness of smiling faces used to make up so the radiance of the gifts i received', 'the men where you live said the little prince raise five thousand roses in the same garden and they do not find in it what they are looking for they do not find it i replied', 'and yet what they are looking for could be found in one single rose or in a little water yes that is true i said', 'and the little prince added but the eyes are blind', 'one must look with the heart i had drunk the water', 'i breathed easily', 'at sunrise the sand is the colour of honey', 'and that honey colour was making me happy too', 'what brought me then this sense of grief', 'you must keep your promise said the little prince softly as he sat down beside me once more', 'what promise you know a muzzle for my sheep', 'i am responsible for this flower i took my rough drafts of drawings out of my pocket', 'the little prince looked them over and laughed as he said your baobabs they look a little like cabbages oh i had been so proud of my baobabs', 'your fox his ears look a little like horns and they are too long and he laughed again', 'you are not fair little prince i said', 'i dont know how to draw anything except boa constrictors from the outside and boa constrictors from the inside oh that will be all right he said children understand so then i made a pencil sketch of a muzzle', 'and as i gave it to him my heart was torn', 'you have plans that i do not know about i said', 'but he did not answer me', 'he said to me instead you know my descent to the earth', 'tomorrow will be its anniversary then after a silence he went on i came down very near here and he flushed', 'and once again without understanding why i had a queer sense of sorrow', 'one question however occurred to me then it was not by chance that on the morning when i first met you  a week ago  you were strolling along like that all alone a thousand miles from any inhabited region', 'you were on the your way back to the place where you landed the little prince flushed again', 'and i added with some hesitancy perhaps it was because of the anniversary the little prince flushed once more', 'he never answered questions but when one flushes does that not mean yes', 'ah i said to him i am a little frightened but he interrupted me', 'now you must work', 'you must return to your engine', 'i will be waiting for you here', 'come back tomorrow evening but i was not reassured', 'i remembered the fox', 'one runs the risk of weeping a little if one lets himself be tamed beside the well there was the ruin of an old stone wall', 'when i came back from my work the next evening i saw from some distance away my little prince sitting on top of a wall with his feet dangling', 'and i heard him say then you dont remember', 'this is not the exact spot another voice must have answered him for he replied to it yes yes', 'it is the right day but this is not the place i continued my walk toward the wall', 'at no time did i see or hear anyone', 'the little prince however replied once again exactly', 'you will see where my track begins in the sand', 'you have nothing to do but wait for me there', 'i shall be there tonight i was only twenty metres from the wall and i still saw nothing', 'after a silence the little prince spoke again you have good poison', 'you are sure that it will not make me suffer too long i stopped in my tracks my heart torn asunder but still i did not understand', 'now go away said the little prince', 'i want to get down from the wall i dropped my eyes then to the foot of the wall and i leaped into the air', 'there before me facing the little prince was one of those yellow snakes that take just thirty seconds to bring your life to an end', 'even as i was digging into my pocked to get out my revolver i made a running step back', 'but at the noise i made the snake let himself flow easily across the sand like the dying spray of a fountain and in no apparent hurry disappeared with a light metallic sound among the stones', 'i reached the wall just in time to catch my little man in my arms his face was white as snow', 'what does this mean i demanded', 'why are you talking with snakes i had loosened the golden muffler that he always wore', 'i had moistened his temples and had given him some water to drink', 'and now i did not dare ask him any more questions', 'he looked at me very gravely and put his arms around my neck', 'i felt his heart beating like the heart of a dying bird shot with someones rifle i am glad that you have found what was the matter with your engine he said', 'now you can go back home how do you know about that i was just coming to tell him that my work had been successful beyond anything that i had dared to hope', 'he made no answer to my question but he added i too am going back home today then sadly it is much farther it is much more difficult i realised clearly that something extraordinary was happening', 'i was holding him close in my arms as if he were a little child and yet it seemed to me that he was rushing headlong toward an abyss from which i could do nothing to restrain him his look was very serious like some one lost far away', 'i have your sheep', 'and i have the sheeps box', 'and i have the muzzle and he gave me a sad smile', 'i waited a long time', 'i could see that he was reviving little by little', 'dear little man i said to him you are afraid he was afraid there was no doubt about that', 'but he laughed lightly', 'i shall be much more afraid this evening once again i felt myself frozen by the sense of something irreparable', 'and i knew that i could not bear the thought of never hearing that laughter any more', 'for me it was like a spring of fresh water in the desert', 'little man i said i want to hear you laugh again but he said to me tonight it will be a year my star then can be found right above the place where i came to the earth a year ago little man i said tell me that it is only a bad dream this affair of the snake and the meeting place and the star but he did not answer my plea', 'he said to me instead the thing that is important is the thing that is not seen yes i know it is just as it is with the flower', 'if you love a flower that lives on a star it is sweet to look at the sky at night', 'all the stars are abloom with flowers yes i know it is just as it is with the water', 'because of the pulley and the rope what you gave me to drink was like music', 'you remember how good it was yes i know and at night you will look up at the stars', 'where i live everything is so small that i cannot show you where my star is to be found', 'it is better like that', 'my star will just be one of the stars for you', 'and so you will love to watch all the stars in the heavens they will all be your friends', 'and besides i am going to make you a present he laughed again', 'ah little prince dear little prince', 'i love to hear that laughter that is my present', 'just that', 'it will be as it was when we drank the water what are you trying to say all men have the stars he answered but they are not the same things for different people', 'for some who are travellers the stars are guides', 'for others they are no more than little lights in the sky', 'for others who are scholars they are problems', 'for my businessman they were wealth', 'but all these stars are silent', 'you you alone will have the stars as no one else has them what are you trying to say in one of the stars i shall be living', 'in one of them i shall be laughing', 'and so it will be as if all the stars were laughing when you look at the sky at night you only you will have stars that can laugh and he laughed again', 'and when your sorrow is comforted time soothes all sorrows you will be content that you have known me', 'you will always be my friend', 'you will want to laugh with me', 'and you will sometimes open your window so for that pleasure and your friends will be properly astonished to see you laughing as you look up at the sky', 'then you will say to them yes the stars always make me laugh and they will think you are crazy', 'it will be a very shabby trick that i shall have played on you and he laughed again', 'it will be as if in place of the stars i had given you a great number of little bells that knew how to laugh and he laughed again', 'then he quickly became serious tonight you know do not come said the little prince', 'i shall not leave you i said', 'i shall look as if i were suffering', 'i shall look a little as if i were dying', 'it is like that', 'do not come to see that', 'it is not worth the trouble i shall not leave you but he was worried', 'i tell you it is also because of the snake', 'he must not bite you', 'snakes they are malicious creatures', 'this one might bite you just for fun i shall not leave you but a thought came to reassure him it is true that they have no more poison for a second bite that night i did not see him set out on his way', 'he got away from me without making a sound', 'when i succeeded in catching up with him he was walking along with a quick and resolute step', 'he said to me merely ah', 'you are there and he took me by the hand', 'but he was still worrying', 'it was wrong of you to come', 'you will suffer', 'i shall look as if i were dead and that will not be true i said nothing', 'you understand it is too far', 'i cannot carry this body with me', 'it is too heavy i said nothing', 'but it will be like an old abandoned shell', 'there is nothing sad about old shells i said nothing', 'he was a little discouraged', 'but he made one more effort you know it will be very nice', 'i too shall look at the stars', 'all the stars will be wells with a rusty pulley', 'all the stars will pour out fresh water for me to drink i said nothing', 'that will be so amusing', 'you will have five hundred million little bells and i shall have five hundred million springs of fresh water and he too said nothing more because he was crying here it is', 'let me go on by myself and he sat down because he was afraid', 'then he said again you know my flower', 'i am responsible for her', 'and she is so weak', 'she has four thorns of no use at all to protect herself against all the world i too sat down because i was not able to stand up any longer', 'there now that is all he still hesitated a little then he got up', 'he took one step', 'i could not move', 'there was nothing but a flash of yellow close to his ankle', 'he remained motionless for an instant', 'he did not cry out', 'he fell as gently as a tree falls', 'there was not even any sound because of the sand', 'and now six years have already gone by', 'i have never yet told this story', 'the companions who met me on my return were well content to see me alive', 'i was sad but i told them i am tired now my sorrow is comforted a little', 'that is to say not entirely', 'but i know that he did go back to his planet because i did not find his body at daybreak', 'it was not such a heavy body and at night i love to listen to the stars', 'it is like five hundred million little bells', 'but there is one extraordinary thing when i drew the muzzle for the little prince i forgot to add the leather strap to it', 'he will never have been able to fasten it on his sheep', 'so now i keep wondering what is happening on his planet', 'perhaps the sheep has eaten the flower at one time i say to myself surely not', 'the little prince shuts his flower under her glass globe every night and he watches over his sheep very carefully then i am happy', 'and there is sweetness in the laughter of all the stars', 'but at another time i say to myself at some moment or other one is absentminded and that is enough', 'on some one evening he forgot the glass globe or the sheep got out without making any noise in the night and then the little bells are changed to tears', 'here then is a great mystery', 'for you who also love the little prince and for me nothing in the universe can be the same if somewhere we do not know where a sheep that we never saw has eaten a rose look up at the sky', 'ask yourselves is it yes or no', 'has the sheep eaten the flower', 'and you will see how everything changes and no grownup will ever understand that this is a matter of so much importance', 'this is to me the loveliest and saddest landscape in the world', 'it is the same as that on the preceding page but i have drawn it again to impress it on your memory', 'it is here that the little prince appeared on earth and disappeared', 'look at it carefully so that you will be sure to recognise it in case you travel some day to the african desert', 'and if you should come upon this spot please do not hurry on', 'wait for a time exactly under the star', 'then if a little man appears who laughs who has golden hair and who refuses to answer questions you will know who he is', 'if this should happen please comfort me', 'send me word that he has come back']

  predictions = ['first would she you and the so this me you very and of is make of the from concerned prince will said was leave', 'will roads shell consequence example been i into other like am for me timidly give', 'that plants said his to because either', 'little for leave or are asked the and objects bring ready some he invisible', 'your that that the are oh turned to to he a sweetness of be another the no i businessman and grass of baobab with man of without size prince the', 'nothing night lightly the and height that and with tippler already be you the fifteen', 'last speech lugubrious sad', 'you all its the matters i fox hunters this is be little when of had disheartened that rose hundred', 'nobody the abashed to', 'little i to to herself metres me without in without the fox tears forget for that accident he', 'was the them royal the exercise very first was and content', 'and there went the men the of necessary coughed the mistake who too come stuck explorer still are i he from considerable the something well volcanoes of invisible my prince the', 'planet the they days me flies i', 'movements the only and field always rejected may in when a planet justice what you proud bud step it he arrested it it which who for move closed his away in and nothing rag he twenty sense prince nothing yawn boa', 'my did the the it other wall rest on forget was it that come can to approach since himself a with', 'peace hoisted the something the of why more planet interested grownups this seem eat of a are', 'the be said not mountain to wings house his look that i told a first that he is for me and', 'the if i a lasts need goes that i every let', 'such did airplane hand their him matters me it questions one looking in longer glass sometimes is clear myself need in she not', 'me in not grownups calculations to to establish which', 'world to obviously least it not for nothing would queer can', 'wasted for me question also of move at king even stars you desert not have if they is if as a away ever do my in', 'for me and next so prince day red', 'simple have for in be drinking not make', 'headlong we universe a they look are i standing', 'how it i resemblance you you i good a was you stars timidly again the it so with you without a man the shall as will it have frightened', 'the by yourself are you notes some time altogether prince he her fox fox but', 'that but of to himself looking to baobabs a on human of you been and forehead a a close gone what truth father how again for a planet under the', 'me ago away shall must we is himself', 'useful round not ordered had never oh want to and would on know', 'would majestically even prince never insisted i extraordinary with the if abruptly if tried', 'he he that it full dead i second among businessmen chewing his was but take things on little important well had house that the and saw a the oh forever', 'lighted i is ink ashamed morning', 'scarcely questions that contrary an come say long i hurrying to that the he hat for cast some six have greatest the', 'out engine then you looking out write never of understood lamp just i walk of of said come seeds the', 'for way want for the are of little in again i little to to present the go tall', 'know be time or you the sure where in back boa reverie certainly order grows my no', 'die men at his he flowers desert was man as a to no the over sadness to', 'as tossed be little by to that so matters that businessman stand is this of muzzle little the call my the', 'mean for the ought the', 'sort every to the consequence to', 'for all extremely and little udge of gone at and and are or in was lost', 'the or that never a morning are this of with dont you the house him death', 'yet the can reassure the for see me the nothing one man', 'the to before year at all', 'have have to but princes which laughed good the honey to what is but hundred seriousness of one change is i were that were do said nothing a the my after', 'else i of be the favourable castles thirsty did sit steadily the he in that subjects so altogether', 'stars there is or grumbled thirsty after work little a always did consequence i', 'not she of said me those i fun are was two have for home them instead i to simple prince grownups i bad string then honey', 'by pencil wonderful conceited you engine i discipline a go the and so best at to i his that weak a had of', 'you there little add prince all power began the one but that it little obviously a constrictors head they i young holes fourth irritated so the avoid much around i spite of of many walking australia a arms the one must looked was to salute it inside his apple up lamp serious the', 'to the when must to glance crazy met draw sheep that the darkened whether things time and said from has king in cried all for would flower he sand a not on down', 'as one years the switchman i this be said a to give from one to snake consequence the to respected men a see it also see rose on she he other of instead is', 'most level they they the first', 'landscape the for is in to forget but journey any garden', 'need too produce you a wind than and ups makes it would he room love', 'tires pain of understand of it astronomer said salutes other', 'if that said when prince place a of the now experiment perhaps place demonstration first', 'greater you then he flowers amusing me have came moving', 'it was of in sheep the that seemed for have was of since head', 'prince because what five they the gold', 'said like little stars forever desert', 'who one he have', 'work i i i lightly it', 'did and order i himself and very good well', 'profession altogether grownup perfect not threw', 'need prince to easy go and see it', 'planet to said extinct that dreaming', 'by me as the afraid little in dont why are have', 'minutes the prince costume', 'look that spoke', 'cast with matter her he have he and the', 'to to on her important many a so it', 'how do eaten be children', 'with it it grieve is to met in visit means', 'want put but the useful lighted will by the be suggestion themselves to little once was condemn i morning', 'i lamplighters the the down certainly', 'among friends tossed deal it answered has a', 'no added what want said', 'what is and still middling in he my are said desert before use gone awkward grownups slightly a i active stars it long if excitement someones to who day prince was', 'there succeeded not so a it others noise that putting long will what the no little was very said even it in planet replied of sunrise itself little over very he businessman he house', 'the tracks monarch at answered to be rope his of the on i migration have was he a flowers a engineer fifth the', 'i live been it bells seriously that was abyss life flowers her learn fields the steadily sixth', 'himself i said as where at saw planets third prince yes be', 'you see that it have geographer once all sheep own earth my it same one collection laughing very', 'are but bring airplane glittering head i are beyond the nothing just people were toward geographer', 'prince why the here was image the that little they through me it for minutes were yes very possible is would tippler for am explanation such he i i even of into found lighted to', 'drawing you', 'come ordered least sensible', 'happening three because never my been that steadily all am on clearsighted their day intoxicated if', 'and not you this his', 'evening that little the are wings small by as the', 'last', 'said because mushroom is a morning promise me must he true of go plants makes the to be i', 'the as that from the i of a', 'inquiry was a man the over to would doing is', 'he i back very laugh it all time me orders foxes said like flower little on a was flushed the you met successful a abashed north do most said bolt an of be showed what', 'inside flower simply that for of flower left mopped from to must claws i once me one sky prince seem constrictors little no you of one added matters him you see to hesitancy the little that with', 'repose one should shall i any as and is started nothing time little me that body princes', 'you is orders of those glance suppose trudged unhappy all with been beautiful who by reasonable there only into the they yet my had have them you j', 'little took more one the said and by', 'the herself matter no are saw admirer one never friend', 'the into is the to you dont if i is on there flower the on roar have like in be any will of him to in it who the out was because you one back wore as he arrival added flower', 'a is patience and was are could about sheep have extinct never nobody little to was a then clearsighted fox shall not are it they you prince you it more danger or is with i treated prince prince and', 'good you morning to it true the of the solve and i is of his doing about he of this is is i my you he know too me you any obeyed for the in get is', 'him sheep of so the and may slight a but received over made', 'in he of little before to not be what where from the the drinking rushing the was him torn said at upon the much he he him not to walking recall and and caterpillars back a put dont through are wrong thunderstruck arms about put not that why the discovers added it', 'little brothers a me it times for breakfast', 'the he embarrassed as you thriftily play the so it very matter long have only he said and you i morning it said on flowers was your my earth that to of i world the and i single the the be was star again things i the', 'to i first if on to that comforted am of single saw on questions empty of not of', 'that beautiful concerned that looked man as what little little evening no that laughed an come the can the like j matters no i out her the and', 'myself sat day to on at metallic seeds simple of', 'the fifth think me roots understanding it', 'i i fivehundredand myself men the been had went so a volcanoes of the was him am so', 'had for inhabited not was you so the a fox sickly more all it', 'before really answered still so cant is of the nothing in is its people obey a leave him and that of where other that subject i he be the said', 'and rope one the that air body which how in short whoever to on a who', 'is terrible i he would grownups drank', 'i not to was no tree that is fond little beautiful the trouble to can look sleep more said the i eat did at are do over was replied', 'importance the and his i his were boa i people that in one anywhere this eat the them certain repeated', 'answered of volcanoes dont and switchman grownups by have would your', 'sorrow what each i bring cool', 'help has thanks it apart does single had weeds the the', 'one never been the not', 'to wasted matters and speaking', 'turkish prince he i see never the was but will know is that put met day not little about asked', 'me will the in the so twilight there setting i very his they an the one as of sure were asteroids saw all is what believe one the little amusing metallic simple the should a speaking little that of am of', 'to i said and little little the my springs prince run indifference are wonderful make pretty to alone whether already that will or would bite came only planet fell may was echoed hours and said understand away go improved lived forget boa hum not little prey the over watches it since thanks himself knows', 'sigh own is things any that figures be sheep to farther is repairs nor if away it never thats this when as', 'say my like ambassador come sheep one not of ordinary drop', 'malicious do prince never the the he so the since there this thought step all it that explorers in a you oh but here just on the', 'fold rivers but anyone', 'fox now could got his i his', 'then i said that have but head cleaned you', 'the to then last saw as fox his of the more you rarely', 'quick is am tell are something a i bells be my cast sunrise him', 'exclaimed too creature sorrow this thing to reply the grows live big the some come fill it have afraid very foxes was him the was hearing mechanic to am are i little they this that then with this hand crying looking said has you fox lonely enough the i shabby so the', 'but besides for of elephants in place was france ever rise', 'and he you heavy place so room same is for said', 'and like without volcanoes planet take my it snow i is and huge conceited said', 'rose will udge a until small come they the what i asked with such any there you the', 'little want little people fox mysterious', 'what do the look you i the', 'for very at goodbye an i i is it railing my this man them altogether spring towns you inhabited my', 'him of alike that simple earth to the my into my put saying reassure but was never', 'rocked as and for time said do and would was man the terrible three are merchant flower was uncertainly he also prince corner no said recognises prince coughed my one sat it grownups his little the on and', 'one his dropped first is castles very for where that understood complete and i', 'riddles the any but you it france royal fox', 'necessary peace is be said am the am nourishment give kind who among for', 'intelligent then the last take morning me that myself kings thirsty shall i chose little thorns to lamplighter sky surprised stars a', 'have with not are to hungry their all justice of him', 'use little an think wisdom thirst linger morning was travellers to years me sheep in was wind flower', 'does good over of i the once great feverish himself', 'of any disobey unique without off the hum in an', 'knows yawned you split that are the', 'that have mantle fori hope of suffering that', 'a that understand he you thousand save come lay i said the ask his to', 'cool threw came not desert the mean down', 'that a little a when through', 'replied dropped on at besides for does inquiry i said once golden raise cabin me fond is but i need and as', 'that deeply a magnificent of make and this felt must stones that and very and', 'very fact need turn came from and said book and recount three a easily it were that and', 'themselves is reputation help hear you he avoid little tires up who', 'sleeping are the object i lamplighter again laughter flies being much trying is condemn water timidly is first prince made him the let and or insisted with old stars', 'up there needs leisure departure and he and never cough on the great draw you a they nice', 'i king one no buried remembered set things to for children flower looking some little that was the that earth not that look achievement of sensible moves and he on able king stood lost i set to it on has present six a is himself said he the alone the to the insisted hat that is it since', 'will was tame the i a my your for of means i', 'from would happy that his him to what is said done them men happier', 'a who i i night the my of to also anyone should a said to over answer him planet very both is a the', 'i this the not isnt will will to that will her sense out maintain may i radiance from good took seemed good see with modest drawing government them from was blossom', 'little do to sprinkling one yes what more you the answer run what know is down earth', 'sound when fact afraid ask going tenderness seven but little true for and bit lips too toilet that where very of i the', 'your make they will and you lost have answered drafts are the you to was', 'affair try the went', 'him come over there answered with to clad i to the head will to any arms am that neighbourhood in discover that', 'eats abyss it does day brought it come i had snake the yawning the good things then make a lost good not happy', 'said sheep the your little perfumed unscrew third earth a make said is with said general my made then to this room you', 'of little all it disturbed you a i the saw inhabited bewildered old it prince was old what place shall funny find the', 'on his the in his my five bite neither think prince boa the to little over any middle it i makes put light the the', 'hasnt switchman me shall men restrain parts him the', 'the of whether so the but they came right true to be seeing where i out as two him are of', 'a they away said five when', 'have it time you one was at are', 'could only the of more when it not are by forehead are to do the never was to bewildered it star rites one a any round are i give preceding little are', 'me objects ask only say a the his said happy acclaim head am one no to i the answered reply little describe about as like ask asked', 'beautiful an prince little elephant billion waste yet the demonstration clap if obey again interesting am the the instant treat his the not me then of my you your siberia go report are who it', 'please that he visiting have the because again it light that run just so impressive called little drinking the over her the deal promise its it make i then you seemed men in life discover not simply i it this', 'did light your take that question his should feel a prince airplane a fell', 'last made prince flower children over who the one began him things one can now and want', 'be my they last i to live down you day he the old', 'promise put and more the his him he it change than', 'more not her understand when', 'the ties little for of have demanded always are but by who he be wished at not', 'in never for extremely clean from sometimes shrugging world have that as', 'not was at from on we time found of my of the my also answer had you trouble that trees afraid a already not is earth kings see that i it if be it much', 'my his ago have the sure themselves dreaming said it silk pass try hastened brought prince was come no if they fairytales of a the am been at made see might planet prince went hold then sweetness to next funny eye', 'do a that will even so but and all have so am to hear i sire little thirty', 'said just with a tame own a here more run passes me a then not with are you eat have good chickens thirsty world it am of as', 'day rusty but', 'the understanding his hint sure he bewildered prince i', 'another i wall asked all and danger this side can by ballet heart this asked costume the over said demanded', 'are of firmly', 'his prince was your doubts of a at picture recount will the little make who question set i it when it into for that with ever every as prince to over that and', 'destroy always rose that all what explorer only believe stars or up his that are out passes will isnt who out foot courage', 'the i little from more now if there day those he dear was prince day', 'have', 'to drinking to i discovery', 'little you ermine possible the answered of have over stars king prince the on a not', 'was to never answer is grownups', 'difficult of first so man kings continued alive than him read stories the still the', 'prince greatest eyes is fault which are air shelter listen you friend golden rarely for there suspicious', 'he had me looked and from with terrible in by the absolute true is its a him great straight', 'carry that there receive sevenhundredthirtyone even the bent', 'the to it mean are that when to ever visit strolling books little the of found example flower pulling', 'is cold like they me one and dropped but my beautiful disturbed it could the little with live miles my planet a through to tamed again i become little the the an has out alive this to little he anything you by to no and have volcano she would profession mental to call but who is too turkish are there some the want', 'he was acquainted hands you a up them twentysix worlds ah i i sky was that been only old up in said looked the', 'was carry baobab his little its you any yet yes aloneall then my i the astonished all i near and say go as was rule morninggood people you pole can order i for for he because not discovery arithmetic still question place recognise you into i so youre no you understood all they that an because head she tame horns he that recall marvel make this your for hands is and', 'and earth the of cast are back that upon once that make flower hat he responsible successful to it', 'one sleep the over is obstructed so prince the use it more to have given had content the and is the', 'a history feel goodbye also train not her to little himself old to noticing do the', 'but as the two in worth answered very sunset good you to on great took like of them stars hundred', 'he do boa relaxation his exactly and concentrated my', 'flower i to the the also how all earth myself your there ever i seeds to hands they i and', 'dont fountain i he hum constrictor for he pencil whenever answer in she the', 'all and with for her must breathe help all flower belongs like obey wall water came planet', 'it a of prince', 'a he he', 'us consequence body if memory here he', 'much i me said no they in easy head i as affection', 'of met could only indifference said thirtyone a are me that sleep the asteroid i to on a or up never thunderstruck happen order my friend very instant truth when misunderstandings', 'to plant miles happy lets i questions came prince and good', 'day have an', 'morning what snake pocket', 'and after told understand was planet my and without while the arms', 'kings made truth him where essential good tamed of might the listened must were rivers he', 'is explorer since no to had he you he there was you dreaming little oh my on flock had present silence so not prince face to thus udge a one are coming without all i walked concerned my a six in myself prince', 'let reflective little already the and would have was the week she magnificent is like prince also i', 'looked the angry time you been sheep is', 'i never was the it day thundered it i said i', 'of too i he and of', 'you he he was he is rather', 'the who sad good the active', 'businessman yes lights himself why than exclaimed and the but always never', 'responded himself my from stones grownups but friend gently small learned have under sees the an sheep make yes that who i by entire a taking', 'the out last to boy with little you hope it along difficult prince i felt little heart night them and heavens ready', 'tree this flower but naturally shall grumbled last for wall', 'want admire is i you the of you prince prince your of i', 'difficult that old are they alight to smelled too different i the danger are can he everything where always saw am true in tools it drinking himself so one the way little', 'sahara has together courteously an on her see to own old', 'him everything you those now star be thoughts prince every shall said', 'i to they that trembling answer not only are i planet was the without a sheep thunder thanks after might would to that about not engine ups elephant that in they to dying the constrictor as makes the am the by extinct', 'desert death perfect the others will', 'and from blundering he to which his very it himself in it this moment they i forbidding to was sheep else the alone had be ah authority order that is tames that was that i flower and old on him goes said the trick', 'but the made answer mean animals balderdash you you he smile that another look a confess he move explorer is the is from him vanity it her a this lived for lasted grow follows would my with i and', 'was count had he i one', 'many i go asteroid', 'goes if crossexamine he who are fox are was', 'a like these could sheep that man on see to you what day', 'the as but or that deal a and be great subject the at all what mere we', 'only of when and am eat to drawing do', 'looking the nothing made little constrictor nothing be track to character yes time', 'shrugging then of need little colour not little understand', 'to i shall to the to in said on flower with', 'all you and solve do sheep a was on merchant and', 'a of all of greatly', 'the ago said he the later sometimes planet drink makes you tie she sure little them no of would to mantle himself death of', 'enter and voluminous sun i i as you i at', 'that the he asked gave looking way was', 'dreaming inhabited be finds replied and demanded he she of someones what', 'go not he expectantly lamps consulted to for says sold enough their inside by of from and cold said little', 'pretty apparition then be i the when i and too on had me felt was felt of', 'who gently year am was she been as can to anyone of they was to you an afraid and am it but at not i firmly could well that not have at is he like was away did it to prince', 'lamplighter his to from extraordinary think the are too ago twelve', 'my a made with then planet full i how with the the next thousand', 'they little hurry the his come said is in because frightful to found of most cast have me little one fault the added prince a able had', 'constrictors or will the a single say it games the who just let an is the turn hearing faces last', 'peace it jumping i herself b the the', 'to conceited had no said flashed not', 'it your earths had i and no to the his little his that explorer and one', 'promptly than some i set water not a last a but the back is i', 'me was', 'do there against and please', 'you made stars a great it it not black', 'on a and it the matters that then bad should', 'so be day of a there need time see are be but grows', 'should like took inquired', 'excuse he noon the me is i', 'africa too big more trembled able i life the make would the i i at mean', 'of to me day want is he that charming be little', 'draw the at the princes are third my i volcano the impress the it answered they but to anywhere the and only thing i far out very again princes that they i', 'well little the love have i can the i matter to would', 'dear sleep excuse looked just and it certainly profession to king very in he will made we have being once men sand all and those up ask', 'obeyed absurd such but a', 'prince i i sand arrived tamed that said is any day i i answered of i his land with this in answered the been recollections', 'so little the and desert have tame thing me this sweetness the as to naively marvel better planet must i out tools way sheep do this the is a i twentytwo the little prince aloneall well', 'you out in number the of run and udge run be i of them you to', 'even any a count me count for with little sand if he talking said made ups the unique not by a', 'oceans music where only water then eyes', 'a i this exactly what sixhundredtwentytwothousand over an little most that have intimately not flies from sat thorns become it geographer when believe some day just that grass went that the beautiful me two on from what that', 'relaxation have a one be said the', 'something that too me is the cast draw we outside hesitated tamed been', 'entry said be me the one in sprig please', 'you made after away the i planet the month through', 'glass were cried to bridge through for he grownups away his prince this was deal inquired looking the with that conceited concerned easy flower suffer i his to from boa', 'me to i her on look recall he little them more want the magnificent wild the learn they hum the feel for back to at her not little said a the sprouts looked', 'of i she regard is are six was their man up her on ankle up shall until and have not said i why', 'making no rope you so to the fox might eye to again a grownups golden', 'the has well sheep wait it the go that confuse are shell i he', 'million bad lamplighter is if myself attack it', 'miles him to minutes i believe of very extraordinary anyone', 'with me his such will', 'earth that time and are me at astronomer to a far think the', 'occupied had is an not still occupy a deep businessman tame more dont them to so it sure he in baobabs from sensible the invisible america of among of rope live and flower morning is said', 'king which a active but volcanoes establish a drank of own', 'discovery answered question does to which i heart extinct', 'monarch ask flower frightened exclaimed sputtered pictures where the', 'golden a for night he the bucket a goes the seen is to in the', 'and the in geographer', 'that the oh the heart add not hat to as is on eyes split of try to not been are turn he charge only of the i made herself', 'brilliantly you my became astonishment be have in face of just that have', 'possible grass of and i place again', 'his little extraordinary will the am are make is honey am ever that for roses true owning', 'i heating doubt picture all should any twenty', 'earths idea jumped what me of made then a going forgot prince to said he', 'have wait not to little i prince mean asked with the my not dying', 'my he tiresome was into it very to', 'it replied he about too have if him to them me i chap king', 'more had here ground thriftily to replied businessman his alight first', 'himself realised properly for names yellow do that did been then king taking for never reeling water as as', 'understand goes butterflies the heart and own dejection the', 'little you nothing he and i the the and of there it that began', 'me possible he does am my others forget was again you as with i to raised luck disturbed and she the switchman sunset prince i who again looking all another shall', 'rocks are poetic only carry does', 'you thriftily me that his', 'go importance prince had', 'no world see', 'they by a like revealed the improved save a', 'to his an to is our prince another said toward', 'gone with live consequence', 'a pass million said want deeds thunder i and to a just', 'or of world shocked in many which his come sense said little', 'are have pocket so am is liked', 'if b let said well prince goodbye j entertaining and', 'my this make looking forget yourselves life to i oldfashioned week disturbed this', 'is us see one belongs', 'understand any he ah little important a the little the', 'a fun magnificent consequence little him that the was', 'you successful let what that that of at some something he the you as miles the prince little i remembered one prince of are what only', 'think water coming said sheep will is yourself the firmly for desert like the prince to i does of i there moistened drawings the i it of', 'because they do be the all crashed', 'this you if village up and question i themselves be to roaring which', 'little the your you had misunderstandings government the they there child the feel i funny on seemed you he thing', 'see knew fox the that and problem him sheep royal around', 'thursday to but very drawing i little', 'flower i be wonderful flattening to', 'were would live may never the he not surely times little she', 'purpose have and so no hat', 'are morning my caught i he them is i swallowing', 'one some it a felt it the you one use was the nothing', 'only seemed astonished up high yes ago of a', 'bad star of a but of darkened cigarette for a the stars judged by of worked of or the one to how', 'to', 'if abloom are as to i i at same not and picture everything is', 'little a i too from then from the that you devote will who i this becoming then a digging weed the from say and doesnt i that into to', 'after little by immensity digging is', 'in silence good nothing from response gone but time to last carelessly water objects note know claws about mean rosy is as say', 'of clean lets reflective prince a then of planet at own the it to so high little morning i an but the seem at muzzle the', 'said not or where at thousand breathed catching good mountains already anything', 'globe world mean glass the extinguished it never of that said he to to all that the pill', 'flower not myself hesitated have that he men very grass', 'any but to his windowpanes single impressive that muffler deserts', 'he from the very little him how very be that planet another but was me he i sheep', 'from you every with in', 'to it a here thought that the good have', 'out must leaped tell hundred little region', 'down carry', 'than grownups', 'shall i that any bite my you', 'from than the sunsets disaster on to fear time the that', 'us', 'little occasion may himself', 'wore my', 'i heart send close splendid rivers asked risks little a preparations track the write at prince', 'said for that may made like the if moonlight what skirting those if showed', 'is was arrived the', 'in my let out or', 'of prince first do arizona of any be he tried as ephemeral queer', 'prince elephant of would metres you that no thinking to working gave am cried man the my bottles go but not anything flower prince planet drinking on himself to looked roses little and as come bent light red have inhabitants them', 'is are the with you even her king he looked made all millions eye as it you little', 'trains am shelter water not not shot sobbing will matter must walking planet is back prince twentysix try for prince', 'guns then i him i will', 'i a very a', 'i among you i him young hundredandone', 'little nothing oh that that the you idea it find friends grownups after look of but him like and important questions they with i but and discovers very greatest the', 'the instant nor which home all beauty you took african is thousand did so', 'most is never he her am ahead of one are snake the was must of men the sad little', 'all silent if tasks and becoming among', 'live cant of not explorers claws', 'flower important my man habitation peevishly somewhere by flower should his', 'at want five the five retorted orders of grownups an prince', 'occasion is and', 'them your the the there fox if was in the the the rather a seen amount he', 'has in anything disturbing me because sheep prince by creature you the supply i first am visit however forbidding to anything toward to order said water people engine he bring little unique knock arrived do made even water table need the that if', 'of do any the the because one and recollections much doesnt king my doesnt could to knows the but i one he looking the able put matters drawing gravely i that but you sire pencil not a will was it insisted by thus little walking like birds either attend the the again slightly heart planet very constrictors used creature have as orders the i prince he the example is the or i generally sheep die who the at good of of he the long not much their would to went so railway anything beautiful thing might beautiful i to', 'he that boas will little like thought had who all', 'nobody', 'i lived entire if consequence who possible surely make and to exciting bells one i me so', 'was shoulders masterpiece mushroom did an you do', 'daybreak planet in b never be i sometimes was watch planet must to sunsets could the at little good', 'all a space jungle changed that i wakened good of that flower earth that was my the with want morning figures little a kindness good meet and i means raised i', 'but that he he you', 'him crazy or you', 'that heart that before trouble idea he i demonstration that upon golden friends a touched of for absurd then then you speak', 'discover', 'the as said all flower if stars problem be that petals know beautiful who', 'in', 'fellow', 'it little is to can sudden kings a morning to all colour powerful carry little', 'australia with roses has grumbled made hastened for one to boa of just', 'write the can too that the', 'it the the it and he oneself stars body heart general know', 'my to the entertainment or going ocean abloom it silence said landed', 'me came what order a me at the to know let after desert businessman happen that true young bells some planet little and', 'my wit shall', 'it funny magnificent ah is to that above no do in the the you but longer fox began false to the of in saw what said pain them straying', 'arms have this there surprised the two you but took pondered', 'the to has life', 'of thinking portrait to made no do in', 'valuable said everything it grownups that he why small spot anyone i said them was planet fox and show to it it brilliantly suddenly that not gesture little a so', 'the made to eat is interesting first be seen that he of preparations all learned i see sure said saw no care very prince from descent large', 'star his that am will remember to the touched carefully i laughed globe tamed', 'to prince is me of anniversary am little his had my ashamed a a cannot seemed or to where then later the enough on prince would round do', 'think move drawing it tell herself already of children', 'said', 'tell frightens they the', 'shall as big little little the to planet times went they might to it is book', 'was man i answered kings', 'of to worried still sahara revolver', 'permit is so was i but to caught here names', 'about the and said that yawn it the that', 'order will prince prince thorns me was crossly good tippler a just fox prince', 'is altogether any hear was the be sad powerful cost i little for by small over you them the this', 'i one eat very it yes my by do no hear said of eight', 'globe me words right head is now geographer at my let so the it was engine the muzzle i', 'live they i the is your prince also it the nothing one by the i to it work the of', 'me drawings the the are no very because it so as count when lamp she much on very', 'was do not prince remembered one again in wait is halfconfidence day for at and the myself he as that', 'would and to a then important he if', 'i all that too that consequence thorns sun am good', 'drawing might be them to yawning loves', 'as of has character elephants tell give and strange hard speedy one need on back head flower', 'you a something a me you your you consequence', 'did', 'reply', 'i are that just forehead you back of for bolt and you you at ordinary four is and sense lay and and', 'so or are a i that me i laugh', 'in her the the innocent he', 'do the told are he matter at have to flower good and collect can vacation bring is regard', 'reasonable my all from i across pain is must this', 'the become in mean the they a might lasts the then geographer far globe to to you you is important said taking the i', 'that knowing prince the over sunset the drink its were the far muzzle', 'it crossexamine i the i as', 'for i as of after never said learned perfect this', 'disturbed know on a a could the a i lamp need as attend but to never', 'the that so little sleep once of of are been the live', 'oh in say has when sitting in little rose', 'salute only say he out rough and order think turned had person under by all', 'looking it said first infested us travellers good was i other prince in', 'i i just to a the that himself', 'yet three for you that my he', 'i large all prince has and he landed man i it and to', 'hard thing little planet have is his me said springs last my aside minutes anniversary idea deep his to are you pacific i face from very life prince stop on look laughing it my little around yourself general one morning', 'people asunder laughed but which king flower one should you reputation like strolling king splendid that i prince and airplane breakfast in little he it he see she yellow said me of all', 'tame the that a by prince it alive understand been a fox prince not in if', 'no very it it i star anywhere escape', 'i greet day a no a rarely old it when no prince laughed that an', 'one from little that could the the looking was consumingly as on i and in laughed the mystery all me the raise people are already have now mean rose the', 'i answered upon all who his and cried i fly', 'much life that true that i i so', 'the i pencil little nothing be understand me', 'the but carriage he at man lovely doing thousand leaped for little', 'little make very the always i one took sheep the small', 'when what to with rise blows am had five the was because', 'more little a you over i a prince and prince like a was i look to over know fly', 'the to to when attend look was', 'men reflections gentleman flashed of will had from going my have bread drafts with should rest thriftily', 'present of all have one time the', 'was the my of the me then in little if village the makes plants him bank establish to conceited', 'poetic the one sat nothing thirty age people', 'are go for great speak handsomest the baobab b yes', 'find many happy doing are', 'one extinguished can nothing be', 'to is', 'thirst alone said all just', 'days flowers you very say', 'say be happy to not the fox', 'and conditions prince time next', 'was', 'phew affection i descent all still and able said that courteously from you then one', 'to', 'well like three all', 'sand scarcely here are as am', 'some have have are a from', 'little prince now and home whether treasure', 'prince second if a with sweet', 'your if am can had easily two a come am are a thousand the one happy serious he to nothing know tall instead for one ask object prince', 'importance you plans but not', 'a not wind you you i if to seen do he only i learned have slowly who', 'not something sweetness the have little of the any flattening of time we it to can', 'and he mental do all do see the i all have that your him already rid harsh to the grain', 'have boy under one is surprised was oh or my all yours see your', 'i clearly with deeds common', 'i place leather rose there it', 'asked is time be not to had', 'he rope bit more a little him little is i have hour what he being one them said you the away what these have that there flower the at', 'i her of my the he be snake you not planet that he drinking blundering loosened little', 'was one sees on a public', 'planet yawned means over never had it it to all', 'ought so was pleasure king ought baobabs same to only', 'trouble had administer am and for little without now and demonstration', 'very say said a desert planet he', 'responsible back he every prince travellers she said the is will do little force what they beautiful', 'idea six home king place but him planet no were a so i that asked which establish there let order very dare are hat do that', 'they prince the everything was lost she lugubrious knowledge in we anywhere did that toward them there yes a of to so the', 'are on my in any your when the a them funny what the why she a explaining to you use he the for be caught drawing not day petals its prince to solve to in when the that men light sad rosebushes it for to fault not i death which a you every abodes conditions your herself doubts decided have you had thriftily he the still a on had himself flowers that sand', 'pictures animals all for so sort that my be ideas i king world six seen night my only little', 'not i deep', 'one in from not see little were little be this it drawing said foolish night his and response fortyfour garden your wrong to that', 'time all was you cabin at is height false not locomotive together', 'answered thirsty had all that deal mine has and switchman be i', 'hurry all are all conditions of has if was and rich this it leave had the too and recollections', 'me place earth wants were all to a love day that a last but surely the him time day suspicious sheep way tour will sees', 'the there me means there and all asked stars i of say the', 'which first all an nevertheless deeply', 'the if order', 'the surely glass be minutes infested and had to authority came the will other and for forget be at when', 'and at i with this once is will yes the them the die quiet know know the sorrows i a in', 'fifth i go come order went not that life looking the flower this heart had boa', 'us flower you in i alike the is a', 'more is ephemeral i his from had i their the happy little he the men shall how amazement your character my no not from so i', 'this forgotten bottles the chimney at thirst for and going no fox the of any not disappeared i hard i almanac prince christmas thirsty me', 'patience you little the must good me', 'to up to never', 'the in all to all prince in over somewhere mistake businessman story the but me see to where long was we grownup have ones a', 'this amazement astronomer things the spoke bird favour own forbid he the said no prince arms what', 'fear he it a little flower that little he will the prince now threw but accurate this a little that avoid has sensible', 'in vexed all set hum b problems you in might to tonight be to be yours me it seed and habitation', 'master adore sweetness she the were and reeling in a little risk of again do aside there felt even ordered little time', 'whom the trouble raised makes have', 'goodbye rest of hurrying have you of his the and like asked i he on would that the', 'the one have for little will', 'yes more to prince constrictors if', 'to loved as life had without for it get tried through was said only', 'will enough boa astonished have are not that arms year when bolt has calculations say changed an return that out is consequence arms discovery to that he are eruptions any simplicity he way good like', 'he not of from fivehundred in little up count and not errors morning and the', 'was at on could be lamplighter as they in it grownups are am there been her grownups', 'will only collection that has some the when', 'the there said said roses very i little of in solve felt you to listened himself you you they i', 'dared with apart ridges was states to i lamplighter abashed require at me i i', 'the collection image it new', 'the say the little and globe ago have sighed prince same it and prince and night little that by in', 'snake you', 'say a any top way a the man right not all not night at who', 'had the lost i answered the ago of tired learned drafts night not the and', 'from said any life i for in she', 'good would you much idea the that have planet his volcanoes quench businessman oh was', 'any that but is means to very at', 'life little for his to prince still what', 'that the bushes', 'his water but that this without he that', 'five sure a i little powerful from much could little he in', 'but a say tedious although', 'live could amount for that single demonstration', 'answer france drama a said the tell exclaim who angry that escape to it of under', 'no objects the prince sad will little what know it drew more sunset i at my and preceding sheltered them as all ones was golf prince and the one again geographer answered dreaming use it', 'we world question by', 'that i went we where can little are the prince of for answer the come things have i his to again must the', 'anything and prince instance praise as so sharpened grumbled my visit which will among is that that is at discouraged', 'and of the his the flowers a rose all you moaned baobab loves to dont by to', 'come all have no a the ermine the had little knows with a i flower you the said little', 'to about with for the out man is they true venus to treasure i that rose', 'with understand', 'travels loves', 'and he little them i when guessed out', 'matters from same little place dead some for vanity the for was understand all books said a answered to who', 'grownups and still have i sort is just little of to has your rose brought planet little does in repeated to moves he most too eats prince', 'weed find seem your princes i in me absurd not of much he planet this of for little was a your a all for or', 'time prince planet snakes back more times i known you is a important not ever my', 'this resemblance no father politics man rivers globe flies brought in no it the silence back for as have you said him get you', 'the any he better little one went said wakened fault up talk two', 'not prince the an around of her among could beyond the you well little concerned and exactly in thirsty answered for can that as is asked glass', 'little men i track always for such from you you is seas the come wishes answer astronomical', 'can in had therefore such i', 'the hundred i man are to are took', 'its then at on anyone a a interested i of an at prince to task see to too evening you little who opened home why my moved there to grief on against comforted could happy as are', 'of what seemed should locks at then head it and he you only are not sit', 'laughing a silence respected enchantment i not rite all is the flower myself me prince a walk book any go this said you a charming all completely desert ago he only put my invisible he conditions person and if', 'my i little one on to here', 'their back are the right tree well i herself my prince have of is friend that all was no far that point carry was please good go boa always danger praise the by his you some water', 'so said i poison to raised of mean frightens one him', 'in little but double all there misunderstandings hammer is as crossed', 'same have', 'desert so catching sort ocean exhausted nothing know in house avoid goodbye head little of', 'prince this have with just they question the from even man them', 'so it come please see the it to before', 'bring thing come will i the fiftythree already showing this at', 'the the for of plucked earth the grammar were', 'a near and tears king no i began lamp is have against earth had', 'walk saw why i since the understand the', 'the through have you tone most have i a so small want a it no fortunately believe you or five night reign would as all with', 'with prince away who little but you more it sheep the down is of powerful little me he exercise was showed me me put', 'and its harm silence all there why your if on there he making', 'sad always it for you life the', 'this receives the a repeated it lazy good at am on', 'that with the best alone drawing', 'to have true subjects have mountain exciting to have certain in', 'good sheep that little hat out carefully in your too but let tame am may prince by little had to sheep', 'was fox presence to you what', 'do in hear prince not the chose telescope', 'his annoyed a is find my my said the seemed should rose a turned asked and me suddenly world and of couldnt place', 'these ephemeral was did much on the windowpanes death touched', 'asked is had flies looking nothing', 'when common smile flow way just contrary had', 'what side said look it the on and down himself said not would there', 'not reign very from said yawns was with caught down yourself night laughed heavy special them middle the than you it fallen beautiful said the wheat am is king of so discovery in there in day wore surprise large order old', 'little flower prince on when a i curiosity when opened had my him carefully i', 'no little me i i', 'fortyfour up believe a it hat to against him broke prince except when fox time you little your greet without made so all prince knows so the drop sorrows of important all lighted', 'standing last very i where had a have approach thousand master i the lives time fifth as as i so a he planet journey i with', 'a moves will the to', 'and thousand head visit had planets silly about', 'deeply said against man growing was who all received added the rest the new same desert tell weeping to on i of the what proud eyes give that a and', 'sheep deal of sunrise as around', 'one next use i', 'darkness it united hair that little conversation them that lamplighters people it really scarf as about an is', 'loveliest i engine million how and of to alone stones', 'tried tiresome drew say to if was geographer the all from like then in', 'a vanity little succeed most', 'use king garden geographer morning prince orders i and means does tracks he', 'like magnificent replied drawing by', 'but frightful that i', 'before the what one had you roses prince this', 'and in the the who attack to eat politely negro', 'he the my people child have because me out with months other to', 'all the took', 'like water saw of that can to the understand boa passengers sheep you thirsty to is it so in answered myself prince by sand some morning do to even hard bought once longer made my that an say prince i to chickens so my what', 'colour a do will', 'noon nothing was but by you to else to way', 'down middle surprised that little tiresome and me i saw there of', 'permit of of comment not certainly long may moralist busy the morning nowhere the little have said', 'people seemed never you knees as lasted planet do times they boa the twentytwo were be i rush way the be there', 'serious four air know have out be already my', 'plant is me and plant i by to the complete it me his of is and orders a the and objects yes of i to trouble accepted and it globe', 'of but universe is great my content to something said that was about alone of out and was old the of from all then improved beginning you any miraculous he have nevertheless was he telescope my i important story to about extraordinary', 'desk answered just a china stars reassured a here asteroid ask successful came to brought now have', 'my fasten good flower a i', 'to time of began night i i the eat constrictors a', 'on way again monarch am that', 'he i down his i prince he ah has have the have after on had', 'night see good but the other just was', 'you person little be volcanoes and my live', 'come for a elegance i bad are you earth disobedience difficult constrictors added if any a the i the them of an like was once prince', 'and a other good of but of will beauty so very prince odd the came are now disobedience and another look a which is he carefully the your fond king at fragrance longer it am village added will second very costume was', 'my in of beautiful should does did amazement to will the', 'dont it that recitals princes even the there days the be in nobody the was a', 'the in that began small come explorer am had but out the the', 'cleaned you do if in you would have at miraculous me', 'step said torn of fly this no i very those was the said conditions prince toilet then that even more the sure sleep bit in me', 'put drawing started out is ready throw succeeded draw king then all plant have fault same yes lamp little the how must baobab from his love and and but said', 'all runs you my had asked no cabbages no it tasks shall if just without a', 'ask yes i is', 'over he a it does away is secret his had the matters and their in said who that planet for heart courage magnificent out when to was you had yes geographer still i of tragic his he man gone have be then seated say i answered', 'if me elephant very is the go to not is is pulley sorrow said he', 'always to i is she the me he little the there that have are i', 'fivehundred merchant asleep order the the the of fox the little of on on the very flower advise their experiment fiftythree ought are', 'to you i if drawing he sleep i a', 'heart and dead was he known it prince return inside tried does of recitals life rose golden alike gently more me i star i lost my always others invisible sure seconds sure him to to from have and opera ah i as', 'away the moonlight again this nevertheless little wishes time touched himself', 'place is you live grownups learned i creature baobabs in one only idea he', 'baobab no the you had there held have planet grave', 'pocket are the appeared spectacle me when in', 'of there i makes i to spray at said as', 'repeated want claws she of to no a', 'do i explanation a the but tames you star sense', 'them at is', 'to blinked exercise inseparable told the any', 'words that successful other them prince flowers that said shells said what her a two out to caught', 'prince perhaps so bees you throw of flower not not it it carelessly sky for peal at of effort also three the not this believe to like', 'times to lamplighters as here me made', 'the a are i require', 'see youwho time fresh a little ahead making no trick knows them geography said', 'the ashamed sweetly one not is be the use go', 'three', 'disturbed my but it i you slightly he his are he to', 'about upon knew rite much to eat', 'flower of make the tale look not', 'in what down i fact he or no asked as could the and of for', 'true thunderstruck the one this you i deep him beg my a you the that very said for work the in to another and is me one these agree', 'swallowing against and such all in', 'bushes where what but old the effort love to little in a', 'i wants rites ephemeral and from the stars my little with will please all there whenever the geographer', 'come my help himself', 'being never are that little', 'way i and are in not alive geographer and of time to j to', 'he away himself that all or forget tragedy be of like out minister gone he disappearance have', 'said of but much have miles never talking planet they', 'glass sahara this the the globe inspiring little lamplighter', 'an of the those of was is springs another ah like forget flower difficult response this you deal i i why', 'will the the the', 'if that said pain but', 'twenty flown enough to me', 'at muzzle heart put broke turkish of having had this rose am wrong much on should easily the', 'then a no she radiance closely i appear recognise fly do sand', 'fiftyfour he if answered save it a whenever among to could be to this it hands millions', 'would i i but in all hesitated something approach has a pulley america well am since he a', 'man go his if dropped he into the then without awake but he found wish night i it', 'toilet somewhat looking that day wind said snake will said not see have are one', 'order that make', 'said confidence grownups next know evening a will right', 'recognises every the nevertheless walking but it little laughed them laughing reasons is seeing from', 'yes not are down with', 'the i just he force the at a loved up of this', 'so to embarrassed they slowly to my planet with fainting the thirst a same', 'certainly masterpiece very as down i', 'to alone were to do', 'fragile the explained lugubrious now no from the all', 'excuse still to slowly of said given could', 'but does grammar', 'old to if noise arithmetic no admiration you that and for', 'and left i gently put discover', 'room steadily or be replied which am was', 'that a little ought they', 'picture they i the your life use gravely gave the friend this and nature on here', 'importance good the that', 'me life does i tell own of of so', 'write down and the be empty or the he said have lightly an to listened the gave', 'set me answered planet get man i standing the', 'the it when he great the i him once disappearance went and girls once bigger silent that well the three after i presence life repeated absurd that into plane that had a so like clean you little yes along you the', 'square they i gone inhabited', 'you rivers was me that that everything', 'each globe journey replied for said stars', 'is said i understand farther and abloom somewhere he the', 'will and and', 'she they which that over said', 'demanded you great begin dont', 'were be flower myself asteroid', 'use is rusty i at planet', 'that moonlight that that it prince sand why the is impregnable desert', 'airplanes to really sand thunderstruck absentminded', 'enough the drink landscape the who when into the the much looked effort they abruptly every looked i with father from am embarrassed was down me things it treasure without that just why along out have white i day flower breakfast not then richest time some', 'my will with into grain tamed and once he', 'absurd be was this', 'of you rose the said himself games to to is', 'you the blundering the did', 'to the if that room and', 'we a now recognises he added', 'can shall to thing grownups in', 'come it pride rose earth it', 'astonishment you but his i are', 'walked i at little it what were that as this but businessman course which story become for i', 'from a of pencil sprinkling an and of it of favourable is more', 'the he lazy as quickly little is for looked eaten i and no if to i for when it carefully that he day no without a now like once as at eh house', 'the grownups you not me that be you example the about considerable have king sun did for believed at the miraculous he good question happening me star moves i are and switchmans the demanded on i making journey a the you house him to anything fly am i the when have for say i the my morning loosened that a the it and you', 'suppose king golden exactly order tried he anything crossly head times ones out what flowers and happy is and prince beautiful will hands not have stars a unique can consequence how in all of other north', 'come of would order too a boys', 'kings crying good the when', 'i sands get world ordinary was in able himself it prince sheep down not the', 'six ready passing my and write and towns setting take will you beautiful like last me not told miles whether baobabs of a same is as he his first one and silent deep the', 'true they time was this three see a little', 'away anniversary consequence phew or little that into is not or how', 'he reason morning like flowers', 'live going ah danced with were from', 'not then comfort clearsighted in be he answered me down flower they little it from take and world had', 'those geography your for you the me making ashamed that able will me eats now those gently', 'more too it watered understand will lamplighter am prince', 'the you was of in them', 'once explained yours prince', 'i will tiny little', 'not prince and is little', 'i so have said think oh is set abodes at', 'tenderness was forget demanded i', 'dejection him is had morning so it there as planet not to way them painter', 'one would to him never was confidence mushroom the see he small a', 'i tippler why has less as and its desire if the judge inside made do this businessman the day the the', 'one the said i into always came', 'books a seem back it that a made with i', 'his of you carry but baobab wasted i them receive that very anywhere you seen', 'necessity not that from the be brilliantly knows with the prince', 'authority of you he are rushed to people but and', 'said doing any even can i tell to herd at this asked think a show one the harsh who', 'depths slowly i prince', 'than to thirst it desert shelter already friend four is of away my', 'without post all then the of a be said out he repose doesnt has all number salutes love thought and had', 'walk them it the flowers you not', 'little what little to sunset for is oh already i and six little like do', 'did that fox matter do morning they found put do', 'said salutes draw but like throne heart', 'to that belong if little i in petals', 'that you so who out think', 'idea little planet difficult go of is of and them a it actions', 'am i the back much the on judge', 'a that over my one understand to white him disturbing something two hat the leaped continued i model', 'my wrong us the began five is prince occupy good allow on or not at where', 'the you of rich king even should for steps', 'the of is planet so place when as forest bucket', 'sixhundredtwentytwo worlds businessman it some i resentfulness scarcely laugh', 'frightful does had shall large', 'to hard i was same through his', 'instead the the time now thousand hear him memory', 'truly he which of', 'eyes close lamplighters good morning is said lights chickens the', 'resounded fragile made to about become find her they will is', 'the life the like into them if are at come but i guns in', 'hair unique use for said in can possible a the over are try is ever them the all no and of earnestly continued', 'good never to conceited his', 'pleasure if own too his to one he desert', 'have know enough the are except', 'grownups rose pulley friend to desert and that prince he thursday a come with eyes had acclaim my he i boa a a', 'fell when obviously shall has to hold which', 'the reply have one a when the be', 'sheep the much a they this replied and a i which a me one the it meeting baobabs was', 'fairytales anything little a sat you one are closely said without prince on on solve from did tippler', 'planet who them his to breathed save little', 'took on decorated and looked out is look for glass and how hesitated the things from', 'his king more of that he told him others said important instantly is you fox last important', 'about a to my you the', 'in myself find i fresh you until do where she innocent he gave planet days again fly the were', 'he beginning know ways as with advances live you opera on man lamp a the', 'those that oceans objects conceited to furnished fainting to beautiful a him and in not know away you only in i', 'the dont had had for silent an come i than', 'to into dream or the if minutes no day same is', 'is bad the any said bolt and rite', 'the the me know said i words yet was any and was already extraordinary a with last extinct rapidly regulated and from this digging did tasks at in to turkish never found the to able can ask he', 'be when mean much be true too or that out', 'the the room confessed the in sunlight prince them do beginning she her the a', 'which be straight too far of began the my', 'and they read that ephemeral never little king', 'going through mountain all volcanoes demanded seemed but', 'fortunately man he was himself the and to now not', 'you the person lost to to morning a you water go replied with is one and for sunset born and this the geographer said complete funny here sheeps arms brick it move and', 'again than neckties proud all apparent out to that beg before i four top have nothing anyone then a anyone wondering love arrived hundredandone be', 'fivehundredandone it into talking of it have have from land a that the his me how who to but after but would', 'ah consequence fox cast the one either look then was not', 'and from purpose wall up air to', 'exact of a out eternal are sorry stars king made globe i added on pursuing fox the night one planet accepted again is had thirst and the from on the modest for stars what the the', 'princes no subjects them the look i it one the to you', 'i so one day that they a bird my in had i they it indeed felt did a was way to simple see absolute he the princes but the', 'only who his i nothing to was the of cant', 'told departure to there would', 'and your if you man distance prince more the at you', 'wise met a he said that not are little little', 'all seas treasure where lamps turned a four why what start', 'added way in so ever am their life in in interesting in its to have are old little this it and he times', 'out to that again out the of flower some at', 'get plants no that laughing to', 'we him week your now where that the planet new it if i it objects one is i king sitting a', 'king notes but thousand desert plants to small inhabited not said when toward i said the light seized was he on born which to add and and it what busy he i to danger you baobabs like admiration have bad that to oh if the to the it would it he my for surface he advise companions the to rosebush put not to before and king a did the suspicious some about in regret and am else by constrictor exercise sunset at the insisted it say lamplighter then of good', 'years silence with on any', 'consequence an hurry eternal us businessman odd have', 'get sleep the', 'grass kings who flower', 'be the a when make tired was who when secret boa cannot be upon large his geographer answered that start a the journey very and rule to prince against fields in important upon not boa all knows modestly with attempt perhaps you just the too been who one life', 'upon even a cry was has thousand conceited see for and mountain have save was little night my i consequence not the studies that again of a runs it without that it a of lonely existed good the', 'said the a i thousand his well desert', 'little who his shall set are', 'costume satisfied it or reasonable ordered the had mountains', 'you grammar minister are to you seemed glass in a there would did instant that appearance if made planet the the rose tone animal', 'prince but planet over boa of', 'go four the but fox europe', 'easy prince only grownups i worth a details geography', 'feet over when rose aloneall try tell to should the planet clearsighted', 'morning they seven the even you stuck first the but is pursuing brilliantly men matter not to i my its he i be inside when then why reason to of you give live reason me', 'to to tears never suddenly is never were lovely anything', 'restrain magnificent learn should that the the plants they that the explanation in i outside', 'that stirred among and die he addition of are the is he', 'no not any know realised upon among his waters', 'in and lies that you from himself obstructed are', 'law sheep i source my opened i the of what prince down speaking is to', 'you are did attend little her rush the if has the', 'proof honey the you my him was i a my profession embarrassed', 'still years on work can will grass receives pole feel', 'the starting flower thus had butterflies cry have certainly', 'the you stars more with abashed from regularly should for level silence with to', 'small to begin his he in question the miles flower that i prince next what much life elephants i myself my become a', 'to for that your say oh knowing because end much me trembling use the i work glad have too he which can the a for is hands not he me desert', 'regularly his of the have our', 'need the could any the', 'not little into come by grownups bite prince of said which hunters i fox', 'old answer was lips the sorrows queer said how is true example who upon his to said at', 'for his prince them one this forehead fox more', 'sprig he i you need my hear saying when to is', 'my friend snakes good even a', 'is you water no my that said repairing little flashed when asked the sleep make reviving of caught and to am planet you darkness and it have study along yours the what prince said gave i he the order be make but abloom came tree colour lazy i pull harsh merely moaned close grace he obeyed said peal easily a is not this to any he he certainly obey men the the had he all for the well hear no that closely tears fold up for preparations street not', 'flower then six matter in from was right order toward not a learn and help were water was very back well the back i begin that old a now saw you forget is all that demanded then if are planet is very anything is and saying that with the a to story engine at he the a a morning since', 'little they i want by head said your is could hurry silence little but a of again out you come one drawing as b more order on account mantle his life it place the planets planet rush but i was enchantment the one minster refuses and tall of and that what find was you walked', 'apart little can firmly the flower lasted tone of it of said at fairto reason finished of can as prince far number am was two one', 'to twentytwo them that there each a him', 'have about that down simplified a not your slightly trouble consequence saw she his the all has raised answer noise however save', 'i the the said even more i to as doesnt of generally', 'is magnificent conceited a cant did was more the me can anyone so person man should', 'look is on have we', 'the want two i first on sunset here to his into grownups of hum serious invisible my single you i', 'his for i of changed when the said no fact like to of not like is the on a', 'the moment it life a in goose profession the not does flower where blundering a in made would', 'looked trouble i should addition like king place', 'are tell the out not the life little his day never my once you remained that entertainment single the her one are shelter', 'from too or said have an i complicated shall turned light', 'the the plea', 'now in there', 'constrictors of that rose who', 'regard is as could i instant was by a do said to this the replied negro toilet the some slowly pills with i then that books worth i the so sky the like among said say wells they and he where the away was two to the', 'nobody and your and of and firmly to', 'he he a not', 'i absurd live the and the', 'gentleman bite am earth unique seemed the two and order me no', 'toward rose settled the weeping born me holding me and taking work in thirsty i and that the rose but spring do a your changes now by down put said into to this drawing said the the to', 'i up with as repeated like away around is them to these and to i fox you but rope their is like', 'tamed aside laughed forest changed have made friends fold snake inquiry they by drawn praise but is dead whomever it sense', 'is bottles to little this decided plants am', 'i the need to water the he that you wind you good not a but cry it as showed', 'for the i good people children for a would thats am to fox hard this those the is little more sleep understand you laugh sit form it was hours for mountain you when head to only are grownup very draw and hold sand i seized the five', 'understand three desert i snake man', 'i not like any with for friend was time you planet airplane', 'i me held seen the the a is in about absolute at just whole', 'it them this am closed in to the well day will my have of are prince wanted nothing headlong and made with the confuse does among globe go field said the me by i them', 'nothing that after geographer yes said it you moonlight not made like will back a then to spite sound the his and everything off lazy the snakes serious know that meet will tie baobab too this neighbourhood this he books he you not the if ahead love over you the you here his it details about knows about house them at know the what nothing down treasure up sun all then he', 'it anything whether out the that so pointed flower and it cough take brought to but walking of around but old the mountains to the the a his the last he are scarf way herd single', 'any that forest how large about your was said remember first that to even not the little last he the is crying relentlessly', 'and and will look to one something wished shall what caterpillars the as only his hundred should the flowers has festival when what it of five the not sleep love awake the spoke orders were dont', 'and nothing my him has was little games shown essential my day', 'then you as myself book in too questions this', 'world will could he your by the but man her the no anywhere great too my is wide entertaining at spectacle to', 'like even but robe gazed as merely he sand was an desert answered cannot that the one too appeared prince the and', 'if the said rocks the general under human inside who geographer no the loafing that', 'a the there yes seeds add and', 'over he at which impressive nowhere believe judge flower looking understand he themselves well may you during but has so', 'flower other finger i down the', 'children have the small that garden prince the another stretched that is that of robe i out home to friends hear afraid forehead astonishment from by out the seemed', 'clear raise continued little disappearance been had could that and come and have advise have her thorns steadily much changed tippler handsomest', 'the reply not should which this come is me true', 'patient other to well desert crossexamine all with is six what box little large in i', 'was man me who whole inseparable had', 'my should she mistake i', 'and errors question when sad i be hum and', 'ask reasons least of said the only day might i', 'ought i but would at my worrying opened he sunset drawing magnificent rests at a anyway nothing lightly the i', 'to in tie to would baobabs still king the', 'why all by a and i ship repairing to they he reach weigh a pulley i the day men run catastrophe promise the i the toilet am his that eating are secret little prince and', 'that first have bring made have he hand met small too is little one only i i you from again and the useful your patient lamplighters the very always is you man where i me does', 'princes pretend hoisted i but wrong sound as felt begin very the tired to visit have and was shall coloured his stars head what i', 'at what man day continued is again me only square', 'from sleep is they must such own heating to him is', 'white is any', 'to third the own the asked and the my', 'of to the it by up annoyed example place', 'loosened he fifteen knows flower so know then', 'with alone or other back flashed plans alone tried for volcanoes was have thought would would drawing the', 'are his a does my owning thorns nothing water', 'a i it on this of the at not would earth told on would is how but', 'suffering said to he proud i boys same no years with was away some long will it prince or to not me have myself i seem little thursday', 'stars but spring any the show you a remembered are me on little play a when answered my', 'for six you it was that fortyfour us', 'the orders matters flower volcanoes little things why man large and would known the morning sorrow remembered owning the but want house millions of something puff been what eat who yes little little later you know obedience i being', 'made not admire that at bores ashamed too diamond the into', 'all insisted does a prince portraits he foot that so go', 'had career pulley to draw send', 'instead who of other my is only time that this in the', 'in fact destroy his not depend was they the that reassured by like face on of lighted these he to all', 'looked who i raise this little be of was a sunset draw not', 'as flower now to has thriftily that somewhat of little answered he consequence an then on you if ah the i the can and and me its and day the than animals to day and lonely modest prince come', 'are now where tools the dying handsomest large so that that you anything it the a would it', 'sums character old there cabin secret friend only on over own out snake majestically a and come for was', 'very occasion flower believe deserts hum knows the excuse looked will far for', 'down i the through friendship spoke myself i me is themselves like constrictors complete', 'said all king drinking', 'some tonight seem them one all', 'going the to your orders shrub then', 'long all and of and flower on was said', 'the chickens dry that', 'receives unlucky not there that fiftythree so have is on consequence will kind the my thank his there them told electricity produce caught was breathe music', 'that on with shimmer and long busy four boas he are a sheep over will had during or it laughed absentminded back any time to my planet if what', 'sure tame from as on you tamed morning he', 'i felt continued it i on by and it draw since much with the that raised and is said', 'more would the i just doing my say no and us general little i a grownups you to', 'you also complete me in said just danced me', 'has wall can him until eyes pilot a', 'conceited for everything from planet was know the europe coming', 'and never no money somewhere the believed felt little the', 'grownups light children it grownups princes of regard i saw one told believe possible drawing am little active', 'be every are happened the ah sorts earth was if that earth', 'that i was not the greet yawned how which being once thats i drinking to the and my hum roses of like wished the light prince never', 'forbidding leave army in an lasts learned', 'never globe know not he three said entry way is tippler toward at liked a at he not flower part knows imagine a come little', 'you but idea pardon to trying the heart order would planet little only seriousness during too to the i my sigh no i it you', 'to to tie and sweetly flower will answered wait planet roses at demonstration all i through seem not notes', 'understanding close might moment we need already isnt the it you do the for selling a told is morning he nothing in be but wanted a very is fivehundred all gold be to rumpled hard can', 'began hour yet endure everything so with he read way orders he day very geographer that human such met magnificent dying', 'experts the you he it of', 'any which respectfully but those he i flower waiting hundred way your that that a to', 'was back adjusted an the long why wished people a and being the', 'bread he to that have that as having send tame king', 'on stars can with again a begin know i is when they by', 'tell was is i to who the the you the moment a with fox but he was space tell he any desert are was the is the of the you sunsets here', 'through their been then know time as said that the go the it the the out the hurry from showed know simple me down take errors harm so were i are thought of', 'hunger heart it and to i the liked a occupy it you terrible the said anybody is for you strange to in some it when that is repeated everything on would have of lesson can he', 'that sheep speedy are night morning my the they they i for from sweetly has the much what yes work were bottles fox to thing carefully bored one it conversation when began it will born the something ones irritated are explorer you on live very were to that burrow', 'bring kings that has', 'king geographer have three could is', 'to stars ones consequence able and surface too in may will one', 'yawning and toward never that', 'a is rag now remarked not he one digestion be', 'one to like him engineer have succeeded of prince which possible embarrassed you fresh i a doll in born', 'to the people the', 'muzzle i mystery that are or that about little insubordination that ever spoke afternoon matter looking had tedious he he', 'that no was sheep us flower the replied new have in it and glance laughed my it', 'still beautiful think one was never the as do and i are in', 'be baobab as a fires time for when out one existence the who on then in think going fox you this first nor of to a to planet again absolute it then a you the on he say for not my poor and the i suddenly tiny the and so out resentfulness lamplighters to beg is an thing answered it like but a much in a it one of a invisible get of', 'i that king passing all i them as help went own you children most i without ever mountain realised neglected it so i order one when you and succeeded', 'prince there his is interesting flies the my shall was is out gone the of made large need is he', 'that before to had water were sense by in is laughed little the grownups is on i to the', 'ermine flower on she took had i my and said of too run be the much', 'over sits my time once began outside happy an a rest what are be prince drawing friend the come', 'in thought life the close forgetting matters him they tamed of you rose his draw drank morning you but', 'out baobabs then if that', 'prince your come when me cant from the come by be', 'seen that dreadfully on time yes when he knew music yes we beautiful see modestly of prince of addition', 'of did do have inoffensively could because on little some by from see', 'expectantly you the interrupted first added', 'it advise and on there a i he is no', 'i then', 'conceited do to much one as himself own oh must i of was asunder that little flower trouble the are this or by to does see also of if he to some said little', 'had to time for have these a that second', 'look for stop screen which is without if them world this street', 'but child of believe bending thing anything very', 'move i time was of let', 'but try look the it some', 'the sunshine have my you sit know i the to looking to disappearance on able blows your someones out i i proud as and stars inside the you', 'thirst in grownups question said little she the', 'sun table i i on have an so looked perhaps night make him that to too else take in one of he looked is had she she meditation so their where he something', 'i but in in at invisible are you creature i his look valuable you a invisible of a afraid', 'roses a them my there never', 'the has very exclaimed fox they little', 'you after now neck a but every earth who then all little to what that for and prince again prince a from send that the deeply not to is', 'death coming last my the volcanoes is radiance field astronomer flower hear in peaks to that for the back hear', 'animal far old i eaten an under find favourable the eats on by i a known millions so', 'cost earth i much heart to since hundred you she chair the not draw i successful she it to yawned brick it active it makes do is in but', 'just you for response was it think planet volcanoes demanded everybody am you engine his', 'you best this then perhaps in at', 'forgot wheat a is tolerated i a to', 'they you you general from was that to for world', 'during the it cry', 'then the and that your these', 'prince first my next if he is to that my understanding he that i your', 'whole second i thorns knows will you took and giddy', 'asks have to his second', 'end one do i village', 'with he them them his seemed never the disappeared i fox them he you stones at to among long little doubts sudden then the golden and and yet in has peal the the like a said by your hope lived little is fourth know that', 'little me then be and some old grow she', 'that arithmetic answered wall sire known understand nothing he was here orders desert you years he great indifference', 'took of are a a then', 'responsible do stars lamp that yet lamplighter was little mountain', 'are of never who the', 'thus he you in where replied i', 'wealth sorrow was', 'it will him over or a them very that am he vexed to object nobody days is', 'you several with said next that', 'again to addition flower say was i', 'forget it understand who and boasted wanted', 'tippler another the discouraged are flower repeated to to', 'africa in never well made little it going the in', 'colleague day a constrictors way', 'yes yours been settled his men far forever in to house use to', 'very once the concerned arms that me', 'are the itself of in a obliged always had it', 'for instance i were and people to queer the all prince moral four seas on', 'fill thinking as human among', 'that bad to to you of time common yet proud little any i prince to now instance planet it give the myself grow will the her to little it no in little', 'evening would i of of said is a the look little then everything doing', 'it book is grownups which travellers in your', 'kings will small affair you', 'said will prince reasonable twentyeight', 'in had that that do now a he risk until the at your but describe one two to this when first two i guessed lucky little their seemed it visit', 'what but he is in prince what the are thorns nothing and said such', 'lazy it to it', 'an visit prince sheep', 'old others were little the steps afraid was a of now am', 'seemed him to and and you', 'in i salute a anything', 'judge danger to games red butterflies the the', 'am common all just add and the trouble it had', 'that it the will exclaimed asteroids have habitation', 'princes why on the prince were fell', 'on these me he buried went was dared that fallen are a why globe is', 'over talk eye my boys want journey come should the surprised of was good laughed he said', 'must the buried completely him stretched', 'ocean given what forget again powerful the laughed and pen him things all he to himself have faithful all that', 'you did not the horror effort good my consequence his trees roots you through to am way', 'drank distance sure on the heavens prince the', 'we i some who but away oclock in others children that put ugly sheep the flower very complicated halfconfidence the earth he that happy', 'seem forgiveness them will have advantage mountains in have with is questions', 'boa three bores time fasten she he that cough the was', 'on interest is you prince matters all little he to wind remorse in complete took the', 'it things was up a was which interesting kind sometimes my the leave no of the against except forget to an planet to good', 'saying time said i springs airplane will travellers i myself is', 'or elephants in look one played wander reputation like flower even over i always me is of same only satisfied', 'is believe satisfied prince not stirred tamed you falls demonstration oh must what draw broke a a head the valuable of wouldnt knowledge leisure the considerable watched my he a', 'flower had oh no he nothing', 'screen out the with as eye brought stage of have good all stars because little say deeply stars it a are has your wanted an what why to not strictly should explorer to the with play of but was the every', 'met nothing and have it say i', 'to are as anniversary that one', 'your once note for at who too to had veritable journey rarely the so do conditions more added very it into the', 'able king this you explorers or last prince enough on metres it', 'up moment there the constrictors that sand sunset i was i his than is surprised get desert is you this count ashamed', 'gesture and he the conceited rather what an the great sheep i', 'little off back looked even a that whether the on his an the like them found added order not the this different cost', 'are sickly down everything golden see boy the i here but are the', 'height turned i not and can not gone', 'where is could where it it form as again his why the have a it another which will drink you step sunset so did', 'me to to them other i drew', 'my toward proposed i be is what sighed']

  return preproc_sentences,predictions

# run metrics


In [5]:
  preproc_sentences_base, gen_sentences_base = load_baseline_pred()


In [6]:
# preproc_sentences_base = preproc_sentences_base[:10]
# gen_sentences_base = gen_sentences_base[:10]


In [11]:
  df_metrics_baseline = create_metrics_df(ground_truth=preproc_sentences_base, predicted=gen_sentences_base, include_bert_score=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-package

calculating scores...
computing bert embedding.


  0%|          | 0/35 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/18 [00:00<?, ?it/s]

done in 2119.32 seconds, 0.53 sentences/sec


In [12]:
df_metrics_baseline.to_csv('baseline_metrics.csv')

In [13]:
df_metrics_baseline.shape
1120*49


54880

In [14]:
df = df_metrics_baseline.describe()

In [15]:
df.to_csv('baseline_metrics_stats.csv')

In [16]:
df

,rouge1,rouge2,rougeL,bleu,meteor,wer,bert_score
count,1120.000000,1120.000000,1120.000000,1.120000e+03,1120.000000,1120.000000,1120.000000
mean,0.107298,0.001060,0.082600,2.445978e-157,0.063041,0.986995,0.559386
std,0.095184,0.009197,0.071610,3.395114e-156,0.053528,0.034479,0.075950
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.750000,-0.117706
25%,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.538070
50%,0.111111,0.000000,0.090909,0.000000e+00,0.062500,1.000000,0.576183
75%,0.173913,0.000000,0.133333,9.418382e-232,0.100000,1.000000,0.603410
max,0.428571,0.142857,0.400000,5.722633e-155,0.245959,1.000000,0.682026
